In [12]:
# Load and read PDF content
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("pdf/DCPR_2034_13-09-2024.pdf")
docs = loader.load()
print("Number of pages in the PDF:", len(docs))

# Extract text from each page for embedding
doc_texts = [page.page_content for page in docs]
#print(" Document  texts",doc_texts)

Number of pages in the PDF: 613


In [13]:
from milvus_model.hybrid import BGEM3EmbeddingFunction
import torch
device = "mps" if torch.backends.mps.is_available() else "cpu"
ef = BGEM3EmbeddingFunction(use_fp16=False, device=device)
dense_dim = ef.dim["dense"]

# Generate embeddings using BGE-M3 model
docs_embeddings = ef(doc_texts)

/Users/krishna/Documents/PROJECTS/pythonproject/Github/chatbotapp/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 179499.46it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 39/39 [01:47<00:00,  2.75s/it]


In [14]:
dense_dim

1024

In [15]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [37]:
# Connect to Milvus given URI
connections.connect(uri="./milvus.db")

# Specify the data schema for the new Collection
fields = [
    # Use auto generated id as primary key
    FieldSchema(
        name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
    ),
    # Store the original text to retrieve based on semantically distance
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=50000),
    # Milvus now supports both sparse and dense vectors,
    # we can store each in a separate field to conduct hybrid search on both vectors
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
]
schema = CollectionSchema(fields)

In [40]:
# Create collection (drop the old one if exists)
col_name = "hybrid_demo"
if utility.has_collection(col_name):
    Collection(col_name).drop()
col = Collection(col_name, schema, consistency_level="Strong")

# To make vector search efficient, we need to create indices for the vector fields
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", sparse_index)
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
col.create_index("dense_vector", dense_index)
col.load()

In [41]:
# For efficiency, we insert 50 records in each small batch
for i in range(0, len(doc_texts), 50):
    # Extract the raw text from Document objects
    batched_texts = [doc.page_content for doc in docs[i : i + 50]]  # Extract the text content
    batched_sparse_vectors = docs_embeddings["sparse"][i : i + 50]
    batched_dense_vectors = docs_embeddings["dense"][i : i + 50]

    # Prepare the batched entities for insertion
    batched_entities = [
        batched_texts,          # Text content
        batched_sparse_vectors, # Sparse vector embeddings
        batched_dense_vectors,  # Dense vector embeddings
    ]

    # Insert the batch into Milvus
    col.insert(batched_entities)

print("Number of entities inserted:", col.num_entities)

Number of entities inserted: 613


In [58]:
# Enter your search query
query = input("Enter your search query: ")
print(query)

# Generate embeddings for the query
query_embeddings = ef([query])
print(query_embeddings)

Our land is a leese land , society formed , can we apply for deem conveyance
{'dense': [array([-0.0215327 ,  0.03360722,  0.00554481, ..., -0.00876445,
       -0.04572418,  0.02149077], dtype=float32)], 'sparse': <Compressed Sparse Row sparse array of dtype 'float64'
	with 17 stored elements and shape (1, 250002)>}


In [59]:
from pymilvus import (
    AnnSearchRequest,
    WeightedRanker,
)

def dense_search(col, query_dense_embedding, limit=5):
    search_params = {"metric_type": "IP", "params": {}}
    res = col.search(
        [query_dense_embedding],
        anns_field="dense_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]

In [60]:
def sparse_search(col, query_sparse_embedding, limit=5):
    search_params = {
        "metric_type": "IP", "params": {}, }
    res = col.search(
        [query_sparse_embedding],
        anns_field="sparse_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]

In [61]:
def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=5,
):
    dense_search_params = {"metric_type": "IP", "params": {}}
    dense_req = AnnSearchRequest(
        [query_dense_embedding], "dense_vector", dense_search_params, limit=limit
    )
    sparse_search_params = {"metric_type": "IP", "params": {}}
    sparse_req = AnnSearchRequest(
        [query_sparse_embedding], "sparse_vector", sparse_search_params, limit=limit
    )
    rerank = WeightedRanker(sparse_weight, dense_weight)
    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=rerank, limit=limit, output_fields=["text"]
    )[0]
    return [hit.get("text") for hit in res]

In [62]:
dense_results = dense_search(col, query_embeddings["dense"][0])
sparse_results = sparse_search(col, query_embeddings["sparse"][[0]])
hybrid_results = hybrid_search(
    col,
    query_embeddings["dense"][0],
    query_embeddings["sparse"][[0]],
    sparse_weight=0.7,
    dense_weight=1.0,
)

In [63]:
print(dense_results)

['Original Notifications shall\nbe referred for perspicuity\n3.11 Notwithstanding anything contained in this regulation,\nrehabilitation project of a slum located on land belonging to\npublic authority and needed for a vital public purpose and\nwhere eligible slum dwellers which cannot be accommodated\nin the in-situ SRS of land under non-buildable reservations, is\ntaken up on an unencumbered plot, TDR as per regulation 32\ntable 12(A) for the area of the land spared for this purpose shall\nbe sanctioned to the owner of the said unencumbered plot and\nthe TDR in lieu of cost of construction of BUA as per sub-\nregulation 4.2 of regulation 32(A) shall be permissible.. For the\npurpose of this Regulation, the BUA shall be as per clause 3.2 of\nthis Regulation. No sale component shall be permissible.\nProvided that the State Govt. or Public authority or a Govt.\nCompany as defined in Sec. 617 of the Companies Act 1956 and\nowned and controlled by the State Govt. (herein after referred\na

In [64]:
print(sparse_results)

['Original Notifications shall\nbe referred for perspicuity\nvest, by provision in a deed of conveyance, in all the owners on\naccount of whose cumulative holdings the LOS is required to be\nkept as LOS in the layout or sub-division/amalgamation/plot of the\nland.\n(b) The proposal for construction of such structure should come as a\nproposal from the owner/owners/society/societies or federation of\nsocieties shall be meant for the beneficial use of the owner/owners/\nmembers of such society/societies/federation of societies.\n(c) Such structures shall not be used for any other purpose, except for\nrecreational activities.\n(d) The remaining area of the LOS shall be kept open to sky and\naccessible to all members as a place of recreation.\n(e) The owner/owners/or society or societies or federation of the\nsocieties shall submit to the Commissioner a registered undertaking\nagreeing to the conditions in (a) to (d) above.\n(f) LOS in a private layout shall be for the exclusive use of the

In [65]:
print(hybrid_results)

['Original Notifications shall\nbe referred for perspicuity\nvest, by provision in a deed of conveyance, in all the owners on\naccount of whose cumulative holdings the LOS is required to be\nkept as LOS in the layout or sub-division/amalgamation/plot of the\nland.\n(b) The proposal for construction of such structure should come as a\nproposal from the owner/owners/society/societies or federation of\nsocieties shall be meant for the beneficial use of the owner/owners/\nmembers of such society/societies/federation of societies.\n(c) Such structures shall not be used for any other purpose, except for\nrecreational activities.\n(d) The remaining area of the LOS shall be kept open to sky and\naccessible to all members as a place of recreation.\n(e) The owner/owners/or society or societies or federation of the\nsocieties shall submit to the Commissioner a registered undertaking\nagreeing to the conditions in (a) to (d) above.\n(f) LOS in a private layout shall be for the exclusive use of the

In [66]:
def doc_text_formatting(ef, query, docs):
    tokenizer = ef.model.tokenizer
    query_tokens_ids = tokenizer.encode(query, return_offsets_mapping=True)
    query_tokens = tokenizer.convert_ids_to_tokens(query_tokens_ids)
    formatted_texts = []

    for doc in docs:
        ldx = 0
        landmarks = []
        encoding = tokenizer.encode_plus(doc, return_offsets_mapping=True)
        tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])[1:-1]
        offsets = encoding["offset_mapping"][1:-1]
        for token, (start, end) in zip(tokens, offsets):
            if token in query_tokens:
                if len(landmarks) != 0 and start == landmarks[-1]:
                    landmarks[-1] = end
                else:
                    landmarks.append(start)
                    landmarks.append(end)
        close = False
        formatted_text = ""
        for i, c in enumerate(doc):
            if ldx == len(landmarks):
                pass
            elif i == landmarks[ldx]:
                if close:
                    formatted_text += "</span>"
                else:
                    formatted_text += "<span style='color:red'>"
                close = not close
                ldx = ldx + 1
            formatted_text += c
        if close is True:
            formatted_text += "</span>"
        formatted_texts.append(formatted_text)
    return formatted_texts

In [67]:
from IPython.display import Markdown, display

# Dense search results
display(Markdown("**Dense Search Results:**"))
formatted_results = doc_text_formatting(ef, query, dense_results)
for result in dense_results:
    display(Markdown(result))

**Dense Search Results:**

Original Notifications shall
be referred for perspicuity
3.11 Notwithstanding anything contained in this regulation,
rehabilitation project of a slum located on land belonging to
public authority and needed for a vital public purpose and
where eligible slum dwellers which cannot be accommodated
in the in-situ SRS of land under non-buildable reservations, is
taken up on an unencumbered plot, TDR as per regulation 32
table 12(A) for the area of the land spared for this purpose shall
be sanctioned to the owner of the said unencumbered plot and
the TDR in lieu of cost of construction of BUA as per sub-
regulation 4.2 of regulation 32(A) shall be permissible.. For the
purpose of this Regulation, the BUA shall be as per clause 3.2 of
this Regulation. No sale component shall be permissible.
Provided that the State Govt. or Public authority or a Govt.
Company as defined in Sec. 617 of the Companies Act 1956 and
owned and controlled by the State Govt. (herein after referred
as the Agency) may undertake Slum Rehabilitation Project on its
own land and be eligible for the benefits under this Regulation
subject to following conditions:
1) The Rehabilitation Project is approved by the SRA.
2) The tenements so constructed in execution of the Project are offered
to slum dwellers located on land belonging to Govt. or Public Authority
and needed for vital public purpose and within 270 days from the date
of issue of LOI the Agency shall identify the slum dwellers.
3) If the Agency fails to identify the slum dwellers needed to be shifted
for a vital public purpose, as above, then the tenements so constructed
shall be offered.
a) to the slum dwellers located on land belonging to Government or Public
Authority within a distance of 2 km. from the land on which the Project
is undertaken, or
b) to the slum dwellers located anywhere in Greater Mumbai on lands
belonging to Govt. or Public Authority, or
4) Further provided that in all above cases the relocation of slum dwellers
in any case will be undertaken not with reference to individuals but
reference to assembly of slum dwellers for the purpose of releasing
the plot of land wholly from slums and not only the patches of land.
Provided that notwithstanding anything mentioned above, project
affected persons due to any vital Public Projects undertaken by
MMRDA including PAP’s under Mumbai Urban Transport Project
(MUTP) being resettled as per the provisions contained in Government
Resolution, Housing and Special Assistance Department, by order no.
700/CR 31/slum-2 dated 12/12/2000 and certified by the Project
Director, MUTP will also be eligible for redevelopment scheme under
this Regulation, as amended from time to time.
228 DCPR 2034


Original Notifications shall Original Notifications shall
be referred for perspicuity be referred for perspicuity
iinn tthhee ssaammee sseeccttoorr oorr ootthheerr sseeccttoorrss wwiitthhiinn tthhee jjuurriissddiiccttiioonn ooff DDRRPP..
1.11 ‘Premium for ownership and terms of lease-. That part of
11..44 PPaavveemmeenntt ddwweelllleerrss aanndd hhuuttmmeenntt ddwweelllleerrss iinn tthhee sslluumm oonn tthhee llaanndd Government/MCGM/MHADA land on which the rehabilitation
rreeqquuiirreedd ffoorr vviittaall uurrggeenntt ppuubblliicc uuttiilliittyy//ppuurrppoossee oorr oonn tthhee hhaazzaarrddoouuss component of the SRS will be constructed shall be leased to the Co-
operative Housing Society of the slum-dwellers for 30 years. Annual
llooccaattiioonn oorr aaffffeecctteedd bbyy DDPP pprrooppoossaallss sshhaallll nnoott bbee rreehhaabbiilliittaatteedd iinn--ssiittuu
bbuutt iinn ootthheerr aavvaaiillaabbllee pplloottss wwiitthhiinn jjuurriissddiiccttiioonn ooff DDRRPP.. lease rent of Rs. 1001 for 4000 per sq. m. of land or part thereof and
lease shall be renewable for a further period of 30 years at a time.
11..55 AA cceerrttiiffiieedd eexxttrraacctt ooff tthhee rreelleevvaanntt eelleeccttoorraall rroollll sshhaallll bbee ccoonnssiiddeerreedd Simultaneously, land under free sale component shall be leased
aaddeeqquuaattee eevviiddeennccee ttoo eessttaabblliisshh tthhee eelliiggiibbiilliittyy ooff aa ppeerrssoonn pprroovviiddeedd hhee directly to the Society/Association of the purchasers of the tenement
iiss ffoouunndd rreessiiddiinngg iinn tthhee ssttrruuccttuurree.. TThhiiss iiss ttoo aavvooiidd tthhee ppoossssiibbiilliittyy ooff under free sale component. Pending the formation of the
ppeerrssoonnss wwhhoo hhaavvee lleefftt tthhee ssttrruuccttuurree ccoommiinngg bbaacckk ttoo ccllaaiimm ffrreeee Society/Association of the purchasers in the free sale component with
tteenneemmeenntt uunnddeerr tthhee sscchheemmee eevveenn tthhoouugghh tthheeyy hhaavvee iinn tthhee nnoorrmmaall a provision for further renewal for a period of 30 years at a time. The
ccoouurrssee lleefftt tthhee sslluumm aanndd ggoonnee aawwaayy iinnttoo aa pprrooppeerr nnoonn--sslluumm aarreeaa oorr oouutt lease rent for the free sale component shall be fixed by SRA.
ooff DDRRPP AArreeaa.. IIff tthhee hhuuttmmeenntt ddwweelllleerrss aarree ffoouunndd rreessiiddeenntt iinn tthhee
ssttrruuccttuurree,, bbuutt tthhee nnaammeess aarree oonn tthhee eelleeccttoorraall rroollll oonn oorr pprriioorr ttoo 11sstt JJaann In addition to above, the Developer/Co-op. Housing Society shall pay
22000000,, aatt aannootthheerr sslluumm//ppaavveemmeenntt ssiittee iinn BBrriihhaann MMuummbbaaii,, tthheeyy sshhaallll bbee premium at the rate of 25% of ASR in respect of SRS proposed to be
ccoonnssiiddeerreedd eelliiggiibbllee bbuutt oonnllyy aatt tthhee ppllaaccee ooff pprreesseenntt rreessiiddeennccee.. IInn ccaassee undertaken on lands owned by Government, Semi-Government
ooff ddoouubbtt oorr ddiissppuuttee,, tthhee ddeecciissiioonn ooff tthhee CCoommppeetteenntt AAuutthhoorriittyy ttoo bbee undertakings and Local Bodies and premium shall go to land owing
aappppooiinntteedd bbyy tthhee GGoovvtt.. iinn HHoouussiinngg DDeeppaarrttmmeenntt sshhaallll bbee ffiinnaall aanndd authority such as MHADA, MCGM, MMRDA as the case may be. The
bbiinnddiinngg oonn aallll tthhee ppaarrttiieess ccoonncceerrnneedd.. premium installment so recovered shall be remitted to concerned land
owing authority within 30 days from the date of recovery.
11..66 AAnn iinnddiivviidduuaall aaggrreeeemmeenntt sshhaallll bbee eenntteerreedd iinnttoo bbyy tthhee ddeevveellooppeerr ssoo
aappppooiinntteedd uunnddeerr DDRRPP bbyy DDRRPP ((SSRRAA)) wwiitthh tthhee eelliiggiibbllee hhuuttmmeenntt ddwweelllleerrss
In the case of Govt. land, the premium shall be deposited in Nivara
ooff eeaacchh ssttrruuccttuurree iinn tthhee sslluumm//ppaavveemmeenntt.. Nidhi.
The amount of premium shall be recovered in installment as may be
11..77 TThhee iinnddiivviidduuaall aaggrreeeemmeenntt eenntteerreedd iinnttoo bbeettwweeeenn hhuuttmmeenntt ddwweelllleerrss aanndd
prescribed by Govt. from time to time. Land owning authority such as
tthhee llaanndd--oowwnniinngg aauutthhoorriittyy//SSRRAA//ddeevveellooppeerr sshhaallll bbee iinn tthhee jjooiinntt nnaammeess
MCGM, MMRDA, MHADA shall not recover land premium in any other
ooff pprraammuukkhh hhuuttmmeenntt ddwweelllleerr aanndd ssppoouussee ffoorr eevveerryy ssttrruuccttuurree..
form. Proposals for SRS on land owned by Central Govt. shall be
accepted only after NOC for the scheme is obtained from the
11..88 HHuuttmmeennttss hhaavviinngg aa pphhyyssiiccaallllyy hhaannddiiccaappppeedd ppeerrssoonn oorr ffeemmaallee--hheeaaddeedd
concerned Central Govt. Department.
hhoouusseehhoollddss sshhaallll bbee ggiivveenn ffiirrsstt pprreeffeerreennccee iinn aalllloottmmeenntt ooff tteenneemmeennttss
ttoo tthhee ootthheerr hhuuttmmeenntt--ddwweelllleerrss.. TThheerreeaafftteerr lloottss sshhaallll bbee ddrraawwnn ffoorr 1.12 Automatic cancellation of Vacant Land Tenure – If any land or part of
aalllloottmmeenntt ooff tteenneemmeennttss ffrroomm tthhee rreemmaaiinniinngg tteenneemmeennttss ttoo tthhee ootthheerr any land on which slum is located is under vacant land tenure the said
tenure/lease created by MCGM or Municipal Commissioner shall stand
hhuuttmmeenntt--ddwweelllleerrss..
automatically terminated as soon as DRP, which is a public purpose,
on such land is prepared and submitted for approval to the DRP cell of
11..99 TTrraannssffeerr ooff PPhhoottoo ppaasssseess –– SSiinnccee oonnllyy tthhee aaccttuuaall ooccccuuppaanntt aatt pprreesseenntt
SRA. Any arrears of dues to be collected by MCGM shall not be linked
wwiillll bbee eelliiggiibbllee ffoorr rreeddeevveellooppmmeenntt,, tthheerree sshhaallll bbee nnoo nneeeedd ttoo rreegguullaarriizzee
to the issue of any certificate or NOC relating to the DRP.
tthhee ttrraannssffeerrss ooff pphhoottoo ppaasssseess tthhaatt hhaavvee ooccccuurrrreedd ssoo ffaarr..
1.13 Recovery of pending dues such as assessment, compensation,
11..1100 AAnnyy ppeerrssoonn wwhhoossee nnaammee iiss eennrroolllleedd iinn aa nnoonn--sslluumm aarreeaa iinn BBrriihhaann
occupational charges, non-agricultural tax/dues etc. pending with
MMuummbbaaii bbuutt hhaass ppuurrcchhaasseedd aa hhuuttmmeenntt iinn DDRRPP aarreeaa aanndd tthheerreeffoorree ggoott
public authorities such as State Govt., MHADA, and/or MCGM shall be
hhiiss nnaammee aallssoo iinncclluuddeedd iinn eelleeccttoorraall rroollll ffoorr tthhee sslluumm aarreeaa,, ii..ee.. hhee hhaass hhiiss
dealt with separately and not be linked to grant of approval or building
nnaammee iinn tthhee eelleeccttoorraall rroollll aatt ttwwoo ppllaacceess,, hhee sshhaallll nnoott bbee eelliiggiibbllee ffoorr tthhee
permission to the DRP.
sscchheemmee..
224444 DCPR 2034 245 DCPR 2034


Original Notifications shall
be referred for perspicuity
vest, by provision in a deed of conveyance, in all the owners on
account of whose cumulative holdings the LOS is required to be
kept as LOS in the layout or sub-division/amalgamation/plot of the
land.
(b) The proposal for construction of such structure should come as a
proposal from the owner/owners/society/societies or federation of
societies shall be meant for the beneficial use of the owner/owners/
members of such society/societies/federation of societies.
(c) Such structures shall not be used for any other purpose, except for
recreational activities.
(d) The remaining area of the LOS shall be kept open to sky and
accessible to all members as a place of recreation.
(e) The owner/owners/or society or societies or federation of the
societies shall submit to the Commissioner a registered undertaking
agreeing to the conditions in (a) to (d) above.
(f) LOS in a private layout shall be for the exclusive use of the residents
of such private layout only and shall not be subjected to acquisition by
MCGM/Appropriate Authority. Further in such cases area of
existing Recreational Open Space shall have to be maintained by
residents of such private layout.
(h) Structures/Uses permitted in layout open spaces:
“Construction of Solid Waste Management System as per the
National Building Code of India, Part 9 Plumbing Services, Section 1-
Water Supply, Drainage & Sanitation (including Solid Waste
Management) paragraph 6 /bio-degradable waste treatment plant,
in the layout LOS, having area 2000 Sq.mt. & above within 10% of the
LOS area.”
(2) LOS in industrial plots/layout of industrial plots in any industrial plot
admeasuring 1000 sq. m or more in area, 15 per cent of the total area
shall be provided as LOS subject to:
(i.) Such LOS shall have proper means of access and shall be so located
that it can be conveniently utilized by the persons working in the
industry;
(ii.) Such LOS shall be kept permanently open to sky and accessible to
all the owners and occupants and trees shall be grown therein at
the rate of 5 trees for every 100 sq. m of the said open space or
114 DCPR 2034


Original Notifications shall
be referred for perspicuity
Provided that when the development is composite and where it is not
possible to hand over part of land to MCGM in such cases entire
area shall be deemed to be marked as existing amenity with the +
sign indicating other uses on the said plot.
20) General conditions to allow development under above regulations:
i) If the area of reservation is not adequate to construct independent
building as mentioned above OR When it is not possible to handover
individual plot along with public amenity, then in such cases Municipal
Commissioner may allow composite building on said land subject to
condition that the built-up area mentioned as above may be allowed
to be handed over to the Planning Authority or Appropriate Authority,
as the case may be, preferably on ground floor and subject to premium
at the rate of 35% and 40% of ASR of the develop land for the zonal
(basic) FSI (In case of suburbs where FSI is 1 it shall be as per ASR of the
developed land and in case of City it shall be 1.33 times of ASR of the
developed land) for AH/R&R and for other reservations respectively or
as may be decided by Government from time to time. If ground floor
is utilised for parking, then the built-up area mentioned above may be
handed over on stilt/first floor with separate entry & exit from public
street. In such cases, built- up area along with proportionate undivided
share of land shall be handed over to the Planning Authority or
Appropriate Authority, as the case may be. In such cases no
compensation of proportionate undivided land share shall be
permissible to the land owner / developer.
ii) It shall be obligatory on Municipal Commissioner to make registered
agreement with the developer /owner at the time of granting the
development permission subject to terms and conditions as it deem
fit. Occupancy Certificate shall be issued only after compliance of all
terms & conditions and getting possession of the constructed
amenity.
iii) The above permissions for development of reservations shall be
granted by the Municipal Commissioner as per the norms mentioned
in these regulations.
iv) The area / built-up area to be handed over to the Planning Authority
under these Regulations shall be earmarked on the sanctioned
building plan clearly mentioning the same, and registered agreement
to that effect shall be executed. After completion of construction, the
said amenity shall be handed over by executing the deed of transfer
88 DCPR 2034


Original Notifications shall Original Notifications shall
be referred for perspicuity be referred for perspicuity
((99)) IInn ccaassee ooff ccoonnvveerrssiioonn//rreevviissiioonn ooff LLOOII aass ppeerr tthhiiss rreegguullaattiioonn aanndd wwhheerree
pplloott iiss rreesseerrvveedd ffoorr ddiiffffeerreenntt//ssaammee ppuubblliicc ppuurrppoossee ffoorr eeqquuaall oorr mmoorree
aarreeaa ooff rreesseerrvvaattiioonn iinn DDPP 22003344 tthhaann DDPP 11999911,, tthheenn ddeevveellooppmmeenntt sshhaallll of MCGM/Appropriate Authority. Proforma of possession receipt shall
bbee aass ppeerr tthhiiss rreegguullaattiioonn.. TThhee DDPP RRooaadd sshhaallll bbee rreecckkoonneedd wwiitthh aass ppeerr be as per Appendix V.
DDPP 22003344..
(6) The area of built-up amenity shall be counted in FSI initially and after
((bb)) SSlluummss iinn IInndduussttrriiaall ZZoonnee ((II))
handing over of said built up amenity the area of built-up amenity shall
((11)) SSlluummss iinn IInndduussttrriiaall ZZoonnee ((II)) sshhaallll bbee aalllloowweedd ttoo bbee rreeddeevveellooppeedd iinn--ssiittuu be allowed free of FSI. Commencement Certificate in respect of BUA in
wwiitthhoouutt ggooiinngg tthhrroouugghh tthhee pprroocceessss ooff cchhaannggee ooff zzoonnee.. IInn tthhee ffrreeee--ssaallee lieu of the built-up amenity handed over to MCGM can be granted only
ccoommppoonneenntt iinn aannyy zzoonnee,, iinn aaddddiittiioonn ttoo rreessiiddeennttiiaall uusseess,, aallll tthhee uusseess after handing over of such built up amenity or before availing zonal
ppeerrmmiitttteedd ffoorr tthhee oorriiggiinnaall zzoonnee sshhaallll bbee ppeerrmmiitttteedd.. FFoorr iinndduussttrriiaall uusseess,, (basic) FSI beyond 75% of gross plot area or before seeking Occupation
tthhee sseeggrreeggaattiinngg ddiissttaannccee sshhaallll bbee mmaaiinnttaaiinneedd ffrroomm tthhee eexxiissttiinngg Certificate to any part of building/ buildings beyond 75% of admissible
iinndduussttrriiaall uunniitt.. BUA, other than the built-up amenity.
((22)) AAnnyy pplloott//llaayyoouutt hhaavviinngg aarreeaa eexxcclluuddiinngg aarreeaa uunnddeerr DDPP RRooaadd//
pprreessccrriibbeedd RRLL aass ppeerr MMMMCC AAcctt mmaayy bbee aalllloowweedd ttoo bbee ddeevveellooppeedd uunnddeerr
(7) The developer shall be entitled to the cost of construction of
tthhiiss RReegguullaattiioonn bbyy iinnssiissttiinngg hhaannddiinngg oovveerr ooff 3355%% aammeenniittyy ooff tthhee
development of buildable amenity as per provision of regulation 17(1)
rreeqquuiirreedd uunnddeerr RReegguullaattiioonn NNoo.. 1144((BB)) ((cc))..
Note (d).
((33)) RReeqquuiirreemmeenntt ooff LLOOSS aass ppeerr tthhee pprroovviissiioonnss ooff RReegguullaattiioonn NNoo.. 2277((11))((aa))
sshhaallll bbee iinnssiisstteedd..
(E) Metro Station inter changes: The buildable reservations if any, on
((44)) IInn tthhee eevveenntt ooff DDPP hhaavviinngg pprroovviiddeedd rreesseerrvvaattiioonn//rreesseerrvvaattiioonnss oonn aa pplloott plots abutting Metro Station interchanges shall cease to exist on
wwhheerree ddeevveellooppmmeenntt uunnddeerr RReegguullaattiioonn NNoo.. 3333((1100)) iiss pprrooppoosseedd,, tthhee
identification of such Metro Station interchanges within 500 m. from
ffoolllloowwiinngg sshhaallll aappppllyy..
such interchange to that extent, subject to condition that 20% of plot
aa.. IIff tthhee aarreeaa uunnddeerr DDPP rreesseerrvvaattiioonn ttoo bbee hhaannddeedd oovveerr ttoo MMCCGGMM area shall be kept reserved for parking lot, which can be developed
((eexxcclluuddiinngg tthhee aarreeaass uunnddeerr DDPP rrooaaddss// rrooaadd sseettbbaacckk)) iiss lleessss tthhaann under accommodation reservation policy under regulation 17(1).
tthhee llaanndd ccoommppoonneenntt ooff bbuuiilltt--uupp aammeenniittyy rreeqquuiirreedd aass ppeerr tthhiiss However, the non- buildable (open spaces) reservations shall be
RReegguullaattiioonn,, oonnllyy tthhee aaddddiittiioonnaall llaanndd aarreeaa sshhaallll bbee pprroovviiddeedd ffoorr implemented to the full extent.
aammeenniittyy..
bb.. IIff tthhee aarreeaa uunnddeerr DDPP rreesseerrvvaattiioonn ttoo bbee hhaannddeedd oovveerr ttoo MMCCGGMM,,
((eexxcclluuddiinngg tthhee aarreeaass uunnddeerr DDPP rrooaaddss// rrooaadd sseettbbaacckk)),, iiss mmoorree tthhaann
tthhee llaanndd ccoommppoonneenntt ooff bbuuiilltt--uupp aammeenniittyy rreeqquuiirreedd aass ppeerr tthhiiss
RReegguullaattiioonn,, tthheenn tthhee pprroovviissiioonn ffoorr aammeenniittyy iiss nnoott nneecceessssaarryy..
((55)) TThhee oowwnneerr sshhaallll ggiivvee aaddvvaanncceedd ppoosssseessssiioonn ooff tthhee llaanndd wwhheerreevveerr
aapppplliiccaabbllee ((ttoo bbee hhaannddeedd oovveerr)) ttoo MMCCGGMM//AApppprroopprriiaattee AAuutthhoorriittyy aatt tthhee
ttiimmee ooff sseeeekkiinngg AApppprroovvaall ffoorr DDeevveellooppmmeenntt ooff pplloott.. IItt sshhaallll bbee
rreessppoonnssiibbiilliittyy ooff tthhee llaanndd oowwnneerr ttoo cclleeaarr aallll tthhee eennccuummbbrraanncceess aanndd
ccoommpplleettee tthhee ffoorrmmaalliittiieess ttoowwaarrddss ttrraannssffeerrrriinngg tthhee llaanndd iinn tthhee nnaammee
110000 DCPR 2034 101 DCPR 2034


In [68]:
# Sparse search results
display(Markdown("\n**Sparse Search Results:**"))
formatted_results = doc_text_formatting(ef, query, sparse_results)
for result in formatted_results:
    display(Markdown(result))


**Sparse Search Results:**

Original Notifications shall
be referred<span style='color:red'> for</span> perspicuity
vest<span style='color:red'>,</span> by provision in<span style='color:red'> a de</span>ed of<span style='color:red'> conveyance,</span> in all the owners on
account of whose cumulative holdings the LOS<span style='color:red'> is</span> required to be
kept as LOS in the layout or sub-division/amalgamation/plot of the<span style='color:red'>
land</span>.
(b) The proposal<span style='color:red'> for</span> construction of such structure should come as<span style='color:red'> a</span>
proposal from the owner/owners/society/soci<span style='color:red'>e</span>ties or federation of
societies shall be meant<span style='color:red'> for</span> the beneficial use of the owner/owners/
members of such<span style='color:red'> society</span>/soci<span style='color:red'>e</span>ties/federation of societies.
(c) Such structures shall not be used<span style='color:red'> for</span> any other purpose<span style='color:red'>,</span> except<span style='color:red'> for</span>
recreational activities.
(d) The remaining area of the LOS shall be kept open to sky and
accessible to all members as<span style='color:red'> a</span> place of recreation.
(<span style='color:red'>e</span>) The owner/owners/or<span style='color:red'> society</span> or societies or federation of the
societies shall submit to the Commissioner<span style='color:red'> a</span> registered undertaking
agreeing to the conditions in (a) to (d) above.
(f) LOS in<span style='color:red'> a</span> private layout shall be<span style='color:red'> for</span> the exclusive use of the residents
of such private layout only and shall not be subjected to<span style='color:red'> </span>acquisition by
MCGM/Appropriate Authority. Further in such cases area of
existing Recreational Open Space shall have to be maintained by
residents of such private layout.
(h) Structures/Uses permitted in layout open spaces:
“Construction of Solid Waste Management System as per the
National Building Code of India<span style='color:red'>,</span> Part 9 Plumbing Services<span style='color:red'>,</span> Section 1-
Water Supply<span style='color:red'>,</span> Drainage & Sanitation (including Solid Waste
Management) paragraph 6 /bio-degradable waste treatment plant<span style='color:red'>,</span>
in the layout LOS<span style='color:red'>,</span> having area 2000 Sq.mt. & above within 10% of the
LOS area.”
(2) LOS in industrial plots/layout of industrial plots in any industrial plot
admeasuring 1000 sq. m or more in area<span style='color:red'>,</span> 15 per cent of the total area
shall be provided as LOS subject to:
(i.) Such LOS shall have proper means of access and shall be so located
that it<span style='color:red'> can</span> be conveniently utilized by the persons working in the
industry;
(ii.) Such LOS shall be kept permanently open to sky and accessible to
all the owners and occupants and trees shall be grown therein at
the rate of 5 trees<span style='color:red'> for</span> every 100 sq. m of the said open space or
114 DCPR 2034<span style='color:red'>
</span>

Original Notifications shall
be referred<span style='color:red'> for</span> perspicuity
of the<span style='color:red'> land</span>-owning authority as having been censused in 1976<span style='color:red'>,</span> 1980<span style='color:red'>,</span>
or 1985 or prior to 1st January<span style='color:red'>,</span> 1995<span style='color:red'>,</span> and 1st Jan 2000.
III Joint ownership with spouse: The reconstructed tenement shall be of
the ownership of the<span style='color:red'> </span>hutment dweller and spouse<span style='color:red'> con</span>jointly<span style='color:red'>,</span> and shall
be so entered and be<span style='color:red'> deem</span>ed to be so entered in the records of the
co- operativ<span style='color:red'>e</span> housing<span style='color:red'> society,</span> including the share certificates or all
other relevant documents.
IV Denotification as Slum Rehabilitation Area: SRA on being satisfied that
it<span style='color:red'> is</span> necessary so to do<span style='color:red'>,</span> or when directed by the State Govt. shall<span style='color:red'>
de</span>notify the Slum Rehabilitation Area.
V Applicability: The following provisions will<span style='color:red'> apply for</span>
redevelopment/construction of accommodation<span style='color:red'> for </span>hutment/
pavement-dwellers through owners/developers/co-operativ<span style='color:red'>e</span> housing
societies of<span style='color:red'> </span>hutment/pavement-dwellers/public authorities such as
MHADA<span style='color:red'>,</span> MIDC<span style='color:red'>,</span> MMRDA etc./Non-Governmental Organizations
anywhere within the limits of MCGM. However<span style='color:red'>,</span> NGO should be
registered under the Maharashtra Public Charitable Trusts Act<span style='color:red'>,</span> 1961
and the Societies Registration Act<span style='color:red'>,</span> 1960 at least<span style='color:red'> for</span> the last five years
should also be got approved by SRA.
VI Right of the Hutment Dwellers:
1.1 Hutment-dwellers<span style='color:red'>,</span> in the slum or on the pavement<span style='color:red'>,</span> eligible in
accordance with the provisions of this Regulation shall in exchange of
the protected dwelling structure<span style='color:red'>,</span> be given free of cost<span style='color:red'> a</span> residential
tenement having<span style='color:red'> a</span> carpet area of 27.88 sq. m including balcony<span style='color:red'>,</span> bath
and water closet<span style='color:red'>,</span> but excluding common areas.
For this purpose of “carpet area” means the net usable floor area
within<span style='color:red'> a</span> tenement excluding that covered by the walls or any other
areas specifically exempted from floor space index computation as per
prevailing Regulation.
1.2 Even those protected dwelling structures having residential areas
more than 27.88 sq. m will be eligible only<span style='color:red'> for</span> 27.88 sq. m of carpet
area where Carpet area means area of tenements exclusive of all areas
under walls including partition walls if any in the tenement.
1.3 All eligible<span style='color:red'> </span>hutment dwellers taking part in the SRS shall have to be
rehabilitated in accordance with the provisions of this Regulation. It
may be in situ and in the same scheme as far as possible.
1.4 Pavement dwellers and<span style='color:red'> </span>hutment dwellers in the slum on<span style='color:red'> land</span>
required<span style='color:red'> for</span> vital public purpose or such location which are otherwise
unsuitable<span style='color:red'> for</span> human<span style='color:red'> </span>habitation or non-suitable due to other
220 DCPR 2034<span style='color:red'>
</span>

Original Notifications shall Original Notifications shall
be referred<span style='color:red'> for</span> perspicuity be referred<span style='color:red'> for</span> perspicuity
Authority in<span style='color:red'> a</span> private existing amenity plot<span style='color:red'>,</span> the existing amenity on
the said plot shall be<span style='color:red'> deem</span>ed to be lapsed. If the use of the existing
PPAARRTT IIIIII LLAANNDD UUSSEESS AANNDD MMAANNNNEERR OOFF DDEEVVEELLOOPPMMEENNTT amenity on the private<span style='color:red'> land is</span> stopped with due permission from<span style='color:red'>
</span>competent authority<span style='color:red'>,</span> then such private<span style='color:red'> land can</span> be developed with
1133.. DDeevveellooppmmeenntt SSttiippuullaattiioonnss..
the special permission of the Commissioner<span style='color:red'> for</span> the permissible<span style='color:red'> land</span>
uses in the said zone.
11.. GGeenneerraall:: IInn eevveerryy ccaasse<span style='color:red'>e</span> ooff ddeevveellooppmmeenntt//rre<span style='color:red'>e</span>--ddeevveellooppmmeenntt ooff aannyy<span style='color:red'>
</span>llaanndd<span style='color:red'>,,</span> bbuuiillddiinngg oorr pprre<span style='color:red'>em</span>miisseess<span style='color:red'>,,</span> tthhee iinntteennddeedd uussee sshhaallll<span style='color:red'> </span>ccoonnffoorrmm ttoo tthhee uussee
If schools which have been developed on unreserved plots and are<span style='color:red'>
</span>zzoonneess<span style='color:red'>,, </span>ppuurrppoossee ooff ddeessiiggnnaattiioonn<span style='color:red'>,, a</span>allllooccaattiioonn oorr rreesseerrvvaattiioonn<span style='color:red'> a</span>ass tthhee ccaasse<span style='color:red'>e</span>
now shown as existing amenity in DP 2034 considering their<span style='color:red'> land</span> use<span style='color:red'>,</span>
mmaayy<span style='color:red'> </span>bbe<span style='color:red'>e,,</span> uunnlle<span style='color:red'>e</span>ssss ssppeecciiffiieedd ootthheerrwwiisse<span style='color:red'>e</span>..<span style='color:red'>
can</span> be redeveloped in future with the additional benefits of FSI
available in DCPR<span style='color:red'>,</span> provided that they shall comply with all other
22.. DDeevveellooppmmeenntt ooff tthhee eexxiissttiinngg<span style='color:red'> a</span>ammeenniittyy//rreesseerrvvaattiioonn//ppaarrttllyy eexxiissttiinngg
regulations of DCPR/ terms & conditions / policy of Govt. Merely by<span style='color:red'>
a</span>ammeenniittyy//ppaarrttllyy rreesseerrvveedd//aammeenniittyy PPlloott::
virtu<span style='color:red'>e</span> of showing the existing<span style='color:red'> land</span> use as existing amenity in respect of<span style='color:red'>
a</span>a)) DDeevveellooppmmeenntt ooff tthhee eexxiissttiinngg<span style='color:red'> a</span>ammeenniittyy::<span style='color:red'> </span>WWhheerre<span style='color:red'>e a</span>a bbuuiillddiinngg oonn<span style='color:red'> a</span>a ssiitte<span style='color:red'>e</span> school will not give the status of authorization unless it<span style='color:red'> is</span> constructed<span style='color:red'>
</span>ccoommpprriisseess<span style='color:red'> a</span>a eexxiissttiinngg<span style='color:red'> a</span>ammeenniittyy<span style='color:red'>,,</span> tthhee ddeevveellooppmmeenntt ooff ssuucchh<span style='color:red'> </span>llaanndd sshhaallll authorisedly as per the prevailing Regulations/ terms & conditions /<span style='color:red'>
</span>nne<span style='color:red'>e</span>cceessssaarriillyy<span style='color:red'> </span>ccoommpprriisse<span style='color:red'>e</span> mmiinniimmuumm BBUUAA eeqquuaall ttoo tthhee eexxiissttiinngg<span style='color:red'> a</span>ammeenniittyy.. AAnnyy policy of Govt.<span style='color:red'>
</span>bbaallaanncce<span style='color:red'>e </span>ppeerrmmiissssiibblle<span style='color:red'>e</span> BBUUAA<span style='color:red'>,,</span> tthheerre<span style='color:red'>e</span>aafftteerr<span style='color:red'>,,</span> mmaayy<span style='color:red'> </span>bbe<span style='color:red'>e</span> ppuutt ttoo uussee iinn<span style='color:red'> </span>ccoonnffoorrmmiittyy
Provided further that if<span style='color:red'> a land is</span> shown as existing amenity and such
wwiitthh ddeevveellooppmmeenntt ootthheerrwwiisse<span style='color:red'>e </span>ppeerrmmiissssiibblle<span style='color:red'>e</span> iinn tthheesse<span style='color:red'>e</span> RReegguullaattiioonnss..
existing amenity<span style='color:red'> is</span> spread over<span style='color:red'> a</span> number of adjoining plots as per
DP 2034 and if any of such plot has not contributed towards its
IInn DDeevveellooppmmeenntt PPllaann iiff pplloott iiss sshhoowwnn<span style='color:red'> a</span>ass eexxiissttiinngg<span style='color:red'> a</span>ammeenniittyy ffoorr<span style='color:red'> a</span>a<span style='color:red'> </span>ppuubblliicc
reservation as per DP 1991 now forming part of existing amenity<span style='color:red'>,
</span>ppuurrppoossee wwiitthh pplluuss ((++)) ssiiggnn<span style='color:red'>,,</span> tthheenn eexxiissttiinngg<span style='color:red'> a</span>auutthhoorriizzeedd BBUUAA wwhhiicchh iiss<span style='color:red'> </span>bbeeiinngg
uusseedd ffoorr tthhee<span style='color:red'> </span>ssaaiidd<span style='color:red'> </span>ppuubblliicc<span style='color:red'> </span>ppuurrppoossee sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e </span>mmaaiinnttaaiinneedd dduurriinngg then such plot will be<span style='color:red'> deem</span>ed to be reserved<span style='color:red'> for</span> such purpose and
rre<span style='color:red'>e</span>ccoonnssttrruuccttiioonn// rreeddeevveellooppmmeenntt ooff<span style='color:red'> </span>ssaaiidd pplloott:: shall be developed as per Regulation No. 17.
PPrroovviiddeedd tthhaatt wwhheerre<span style='color:red'>e</span> tthhee CCoommmmiissssiioonneerr wwiitthh ssppe<span style='color:red'>e</span>cciiaall wwrriitttteenn Exception: Traffic Islands constituting part of the road may be<span style='color:red'>
</span>ppeerrmmiissssiioonn<span style='color:red'>,,</span> ddeecciiddeess tthhaatt tthhee bbuuiillddaabblle<span style='color:red'>e</span> eexxiissttiinngg<span style='color:red'> a</span>ammeenniittyy iiss nnoo lloonnggeerr merged with carriageway in order to improve the traffic movement
rre<span style='color:red'>e</span>qquuiirreedd iinn vviie<span style='color:red'>e</span>ww ooff tthhee<span style='color:red'> a</span>avvaaiillaabblle<span style='color:red'>e a</span>ammeenniittyy iinn tthhee vviicciinniittyy<span style='color:red'>,,</span> tthheenn tthhee with the special written permission of Commissioner. (Unless these<span style='color:red'>
a</span>ammeenniittyy sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> ddeevveellooppeedd ffoorr ootthheerr bbuuiillddaabblle<span style='color:red'>e </span>ppuubblliicc<span style='color:red'> </span>ppuurrppoossee are shown in open space calculation)<span style='color:red'>
</span>ccoonnssiiddeerriinngg tthhee ddeeffiicciieennccyy iinn tthhee aaddmmiinniissttrraattiivve<span style='color:red'>e</span> wwaarrdd::
b) Development of Reservations: Land reserved in the DP shall be
PPrroovviiddeedd ffuurrtthheerr tthhaatt iiff<span style='color:red'> a</span>a<span style='color:red'> </span>llaanndd iiss sshhoowwnn<span style='color:red'> a</span>ass eexxiissttiinngg<span style='color:red'> a</span>ammeenniittyy<span style='color:red'> </span>ppuurrssuuaanntt developed as per the provisions of Regulation No.17<span style='color:red'>,</span> Table No.3<span style='color:red'>,</span> 4
ttoo vvaalliidd ddeevveellooppmmeenntt<span style='color:red'> </span>ppeerrmmiissssiioonn ((eexxcce<span style='color:red'>e</span>pptt iinn ccaasse<span style='color:red'>e</span> ooff PPPPLL)),<span style='color:red'>, a</span>ass<span style='color:red'> </span>ppeerr LLOOII & 5<span style='color:red'> for land</span> use and manner of development.
iissssuu<span style='color:red'>e</span>edd// ggrraanntteedd uunnddeerr tthhee pprriinncciipplle<span style='color:red'>e</span> ooff AARR//II ttoo<span style='color:red'> </span>RR && CC<span style='color:red'>,,</span> pprriioorr ttoo<span style='color:red'> </span>ccoommiinngg
c) Development of<span style='color:red'> land</span> partly shown as of existing amenity: Where<span style='color:red'> a</span>
iinnttoo ffoorrcce<span style='color:red'>e</span> ooff tthheesse<span style='color:red'>e</span> RReegguullaattiioonnss<span style='color:red'>,,</span> aanndd wwhheenn ssuucchh<span style='color:red'> a</span>ammeenniittyy iiss nnoott
building exists on<span style='color:red'> a</span> site shown as existing amenity in the DP<span style='color:red'>,</span> only its
ddeevveellooppeedd//uunnddeerr ddeevveellooppmmeenntt oonn ssiitte<span style='color:red'>e,,</span> tthheenn<span style='color:red'> a</span>att tthhee ooppttiioonn ooff oowwnneerr
appropriate part of<span style='color:red'> land</span> as used<span style='color:red'> for</span> such existing amenity shall be
//ddeevveellooppeerr<span style='color:red'>,,</span> tthhee<span style='color:red'> </span>llaanndd uunnddeerr rreeffeerreenncce<span style='color:red'>e</span> sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> ddeevveellooppeedd<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthhee
used<span style='color:red'> for</span> the said purpose and the remaining part of the<span style='color:red'> land</span>/building
ddeevveellooppmmeenntt<span style='color:red'> </span>ppeerrmmiissssiioonn ggrraanntteedd oorr mmaayy<span style='color:red'> </span>bbe<span style='color:red'>e</span> ddeevveellooppeedd uunnddeerr tthheesse<span style='color:red'>e</span>
of the developable<span style='color:red'> land</span> may be put to use in conformity with the
rreegguullaattiioonnss<span style='color:red'> </span>ccoonnssiiddeerriinngg rreesseerrvvaattiioonn<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthheenn<span style='color:red'> </span>DDPP.. TThhee
purpose of development as otherwise permissible in these
CCoommmmiissssiioonneerr wwiitthh tthhee ssppe<span style='color:red'>e</span>cciiaall wwrriitttteenn<span style='color:red'> </span>ppeerrmmiissssiioonn mmaayy ddeecciidde<span style='color:red'>e</span> tthhaatt tthhee
Regulations.<span style='color:red'>
</span>ssaaiidd<span style='color:red'> a</span>ammeenniittyy iiff nnoo lloonnggeerr rre<span style='color:red'>e</span>qquuiirreedd iinn vviie<span style='color:red'>e</span>ww ooff tthhee<span style='color:red'> a</span>avvaaiillaabblle<span style='color:red'>e a</span>ammeenniittyy iinn
tthhee vviicciinniittyy<span style='color:red'>,,</span> tthheenn tthhee<span style='color:red'> a</span>ammeenniittyy sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> ddeevveellooppeedd ffoorr ootthheerr<span style='color:red'> </span>ppuubblliicc d) Development of Amenity plot: Wherever possession of<span style='color:red'> land is
</span>ppuurrppoossee<span style='color:red'> </span>ccoonnssiiddeerriinngg tthhee ddeeffiicciieennccyy iinn tthhee aaddmmiinniissttrraattiivve<span style='color:red'>e</span> wwaarrdd.. obtained by the Planning or Appropriate Authority as an amenity plot
as per the provisions of these Regulations<span style='color:red'>,</span> such entire plot shall be
PPrroovviiddeedd ffuurrtthheerr tthhaatt<span style='color:red'>,,</span> oonn ssuurrrreennddeerr ooff tteennaannccyy<span style='color:red'> </span>bbyy MMCCGGMM//AApppprroopprriiaatte<span style='color:red'>e</span>
4422 DCPR 2034 43 DCPR 2034<span style='color:red'>
</span>

Original Notifications shall Original Notifications shall
be referred<span style='color:red'> for</span> perspicuity be referred<span style='color:red'> for</span> perspicuity
Rg = Rate<span style='color:red'> for land</span> in Rs. per sq. m. as per ASR of generating plots in<span style='color:red'>
</span>WWhheerre<span style='color:red'>e,,</span> generating year
AA==<span style='color:red'> </span>ccoosstt ooff<span style='color:red'> </span>ccoonnssttrruuccttiioonn ooff<span style='color:red'> a</span>ammeenniittyy iinn rruuppeeeess<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthhee<span style='color:red'> </span>rraatteess ooff Rr = Rate<span style='color:red'> for land</span> in Rs. per sq. m. as per ASR of receiving plot in<span style='color:red'>
</span>ccoonnssttrruuccttiioonn<span style='color:red'> </span>mmeennttiioonneedd iinn AAnnnnuuaall SSttaatteemmeenntt ooff<span style='color:red'> </span>RRaatteess ((AASSRR)) pprre<span style='color:red'>e</span>ppaarreedd generating year Y= TDR<span style='color:red'> de</span>bited from DRC in sq. m.<span style='color:red'>
</span>bbyy tthhee IInnssppe<span style='color:red'>e</span>ccttoorr GGeenneerraall ooff RRe<span style='color:red'>e</span>ggiissttrraattiioonn ffoorr tthhee yye<span style='color:red'>e</span>aarr iinn wwhhiicchh
Note: - All TDR including slum and heritage TDR shall be utilized<span style='color:red'>
</span>ccoonnssttrruuccttiioonn ooff<span style='color:red'> a</span>ammeenniittyy iiss<span style='color:red'> </span>ccoommmmeenncceedd.. as per this regulation only.
BB<span style='color:red'> </span>==<span style='color:red'> </span>llaanndd<span style='color:red'> </span>rraatte<span style='color:red'>e </span>ppeerr ssqq..mm..<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthhee AAnnnnuuaall SSttaatteemmeenntt ooff<span style='color:red'> </span>RRaatteess ((AASSRR))
5.4 Utilization of TDR:
pprre<span style='color:red'>e</span>ppaarreedd<span style='color:red'> </span>bbyy tthhee IInnssppe<span style='color:red'>e</span>ccttoorr GGeenneerraall ooff RRe<span style='color:red'>e</span>ggiissttrraattiioonn ffoorr tthhee yye<span style='color:red'>e</span>aarr iinn wwhhiicchh<span style='color:red'>
</span>ccoonnssttrruuccttiioonn ooff<span style='color:red'> a</span>ammeenniittyy iiss<span style='color:red'> </span>ccoommmmeenncceedd.. 5.4.1 Utilization of Transferable Development Rights (TDR) and Road
Width Relation will be governed by regulation 30 (A) subject to
BBUUAA<span style='color:red'> </span>== BBuuiilltt--uupp<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa ooff<span style='color:red'> </span>ccoonnssttrruucctteedd//ddeevveellooppeedd<span style='color:red'> a</span>ammeenniittyy
following notes:
PPrroovviiddeedd tthhaatt iinn ccaasse<span style='color:red'>e</span> SSlluumm RReeddeevveellooppmmeenntt SScchheemme<span style='color:red'>e</span> uunnddeerr<span style='color:red'> </span>ccllaauusse<span style='color:red'>e</span> 33..1111
Note: -
ooff RReegguullaattiioonn 3333((1100)) tthhee CCoonnssttrruuccttiioonn AAmmeenniittyy TTDDRR sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> iinnccrre<span style='color:red'>e</span>aasseedd
i) The maximum permissible TDR that<span style='color:red'> can</span> be utilised on any plot shall<span style='color:red'>
</span>bbyy 11..3355<span style='color:red'> </span>ttiimmeess tthhee TTDDRR<span style='color:red'> </span>ggeenneerraatteedd<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr<span style='color:red'> a</span>abboovve<span style='color:red'>e</span> ffoorrmmuullaa..
be as per regulation 30(A) and/or as specified in these Regulations.
55..00 UUTTIILLIISSAATTIIOONN TTRRAANNSSFFEERRAABBLLEE DDEEVVEELLOOPPMMEENNTT<span style='color:red'> </span>RRIIGGHHTTSS ((TTDDRR))::----
ii) Maximum permissible TDR loading as mentioned above on any plot
55..11 AA hhoollddeerr ooff DDRRCC wwhhoo ddeessiirreess ttoo uussee FFSSII<span style='color:red'> </span>ccrreeddiitt tthheerreeiinn oonn<span style='color:red'> a</span>a ppaarrttiiccuullaarr
shall be exclusive of FSI allowed<span style='color:red'> for</span> inclusive housing if any.
pplloott ooff<span style='color:red'> </span>llaanndd sshhaallll<span style='color:red'> a</span>attttaacchh vvaalliidd DDRRCCss ttoo tthhee eexxtteenntt rre<span style='color:red'>e</span>qquuiirreedd wwiitthh hhiiss<span style='color:red'>
a</span>apppplliiccaattiioonn ffoorr ddeevveellooppmmeenntt<span style='color:red'> </span>ppeerrmmiissssiioonn.. PPrrooppoossaall ffoorr TTrraannssffeerraabblle<span style='color:red'>e</span> iii) The quantum of maximum permissible TDR loading mentioned above
DDeevveellooppmmeenntt RRiigghhttss ((TTDDRR)) uuttiilliissaattiioonn sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> ssuubbmmiitttteedd<span style='color:red'> a</span>alloonngg wwiitthh tthhee shall include slum TDR at least 20 % and maximum to the extent of
ddooccuummeennttss<span style='color:red'> a</span>ass mmaayy<span style='color:red'> </span>bbe<span style='color:red'>e</span> pprreessccrriibbeedd<span style='color:red'> </span>bbyy tthhee CCoommmmiissssiioonneerr oorr<span style='color:red'> </span>bbyy tthhee 50% of column no. 6 of Table No. 12 regulation 30(A) or as decided by
Govt. time to time. Slum TDR as per this regulation and DRC
GGoovveerrnnmmeenntt ffrroomm<span style='color:red'> </span>ttiimme<span style='color:red'>e</span> ttoo<span style='color:red'> </span>ttiimme<span style='color:red'>e</span>..
generated from the vary said<span style='color:red'> land</span> and/or DRC generated from other
55..22 WWiitthh aann<span style='color:red'> a</span>apppplliiccaattiioonn ffoorr ddeevveellooppmmeenntt<span style='color:red'> </span>ppeerrmmiissssiioonn<span style='color:red'>,,</span> wwhheerre<span style='color:red'>e</span> aann oowwnneerr
location may be utilized up to the permissible limit mention above.
sseeeekkss uuttiilliissaattiioonn ooff DDRRCC<span style='color:red'>,,</span> hhee sshhaallll ssuubbmmiitt tthhee DDRRCC ttoo tthhee MMuunniicciippaall
CCoommmmiissssiioonneerr wwhhoo sshhaallll eennddoorrssee tthheerre<span style='color:red'>e</span>oonn iinn wwrriittiinngg iinn<span style='color:red'> </span>ffiigguurreess aanndd iv) If<span style='color:red'> a</span> plot<span style='color:red'> is</span> situated on access road having dead end within 50 mt. from
wwoorrddss<span style='color:red'>,,</span> tthhee qquuaannttuumm ooff tthhee TTDDRR pprrooppoosseedd ttoo<span style='color:red'> </span>bbe<span style='color:red'>e</span> uuttiilliisseedd<span style='color:red'>,, </span>bbeeffoorre<span style='color:red'>e</span> the main road<span style='color:red'>,</span> having minimum<span style='color:red'> </span>width of 9m or more then such plot<span style='color:red'>
</span>ggrraannttiinngg ddeevveellooppmmeenntt<span style='color:red'> </span>ppeerrmmiissssiioonn.. shall be treated as fronting on main road<span style='color:red'> for</span> the purpose of<span style='color:red'>
</span>utilisation of TDR. Similarly<span style='color:red'>,</span> if the plot derives from 9m wide internal
road then such plots also eligible<span style='color:red'> for</span> the purpose<span style='color:red'> for </span>utilisation of
55..33 TThhee TTrraannssffeerraabblle<span style='color:red'>e</span> DDeevveellooppmmeenntt RRiigghhttss ((TTDDRR))<span style='color:red'> </span>ggeenneerraatteedd ffrroomm aannyy<span style='color:red'> </span>llaanndd
TDR.
uussee<span style='color:red'> </span>zzoonne<span style='color:red'>e</span> sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> uuttiilliisseedd oonn aannyy rre<span style='color:red'>e</span>cce<span style='color:red'>e</span>iivviinngg pplloott iirrrreessppe<span style='color:red'>e</span>ccttiivve<span style='color:red'>e</span> ooff tthhee
v) The relaxation premium<span style='color:red'> for</span> the use of slum TDR i.<span style='color:red'>e</span>.10% of normal<span style='color:red'>
</span>llaanndd uussee<span style='color:red'> </span>zzoonne<span style='color:red'>e</span> aanndd aannyywwhheerre<span style='color:red'>e</span> iinn MMuummbbaaii CCiittyy<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa ((iissllaanndd<span style='color:red'> </span>cciittyy)) aanndd
premium shall be charged while<span style='color:red'> con</span>doning<span style='color:red'> de</span>fici<span style='color:red'>e</span>ncies in open
MMuummbbaaii SSuubbuurrbbaann //EExxtteennddeedd SSuubbuurrbbaann<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa.. TThhee eeqquuiivvaalleenntt
spaces.
qquuaannttuumm ooff TTrraannssffeerraabblle<span style='color:red'>e</span> DDeevveellooppmmeenntt RRiigghhttss ((TTDDRR)) ttoo<span style='color:red'> </span>bbe<span style='color:red'>e </span>ppeerrmmiitttteedd
oonn rre<span style='color:red'>e</span>cce<span style='color:red'>e</span>iivviinngg pplloott sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> ggoovveerrnneedd<span style='color:red'> </span>bbyy tthhee ffoorrmmuullaa<span style='color:red'> </span>ggiivveennbbeellooww:: --
5.4.2 The restrictions of total maximum permissible built-up area in terms
FFoorrmmuullaa:: XX<span style='color:red'> </span>== ((RRgg //<span style='color:red'> </span>RRrr)) xx<span style='color:red'> </span>YY
of FSI with respect to road<span style='color:red'> </span>width mentioned in these Regulation<span style='color:red'>,</span> shall<span style='color:red'>
</span>WWhheerre<span style='color:red'>e,,</span> XX<span style='color:red'> </span>== PPeerrmmiissssiibblle<span style='color:red'>e</span> UUttiilliissaattiioonn ooff TTDDRR//DDRR iinn ssqqmm oonn rre<span style='color:red'>e</span>cce<span style='color:red'>e</span>iivviinngg pplloott
not be applicable in cases where<span style='color:red'>,</span> the permissible FSI<span style='color:red'> is</span> more than
114422 DCPR 2034 143 DCPR 2034<span style='color:red'>
</span>

Original Notifications shall Original Notifications shall
be referred<span style='color:red'> for</span> perspicuity be referred<span style='color:red'> for</span> perspicuity
3333((77))((BB)) AAddddiittiioonnaall FFSSII ffoorr RReeddeevveellooppmmeenntt ooff eexxiissttiinngg developed as per Regulation No 33(7)(A) and remainder notional plot
rreessiiddeennttiiaall hhoouussiinngg ssoocciieettiieess eexxcclluuddiinngg bbuuiillddiinnggss<span style='color:red'> </span>ccoovveerreedd shall be developed as per this Regulation.
uunnddeerr rreegguullaattiioonn 3333((77)) aanndd 3333((77))((AA))::
33(8) [ Construction of Affordable Housing in Special Development Zone (SDZ)
11)) IInn ccaasse<span style='color:red'>e</span> ooff rreeddeevveellooppmmeenntt ooff eexxiissttiinngg rreessiiddeennttiiaall hhoouussiinngg ssoocciieettiieess
eexxcclluuddiinngg bbuuiillddiinnggss<span style='color:red'> </span>ccoovveerreedd uunnddeerr rreegguullaattiioonn 3333((77)) aanndd 3333((77))((AA)) (I) Private<span style='color:red'> land</span> following in SDZ
pprrooppoosseedd<span style='color:red'> </span>bbyy HHoouussiinngg ssoocciieettiieess//llaannddlloorrddss oorr tthhrroouugghh tthheeiirr pprrooppoonneennttss (A) General
The provision of this Regulation shall<span style='color:red'> apply</span> to any contiguous<span style='color:red'>,</span> unbroken and
wwhheerre<span style='color:red'>e</span> eexxiissttiinngg<span style='color:red'> </span>mme<span style='color:red'>em</span>mbbeerrss<span style='color:red'> a</span>arre<span style='color:red'>e</span> pprrooppoosseedd ttoo<span style='color:red'> </span>bbe<span style='color:red'>e</span> rre<span style='color:red'>e</span>--<span style='color:red'> a</span>accccoommmmooddaatteedd oonn tthhee
uninterrupted piece of<span style='color:red'> land,</span> not less than 2.0 ha<span style='color:red'>,</span> and not disqualified from<span style='color:red'>
</span>ssaamme<span style='color:red'>e</span> pplloott<span style='color:red'>,,</span> iinncceennttiivve<span style='color:red'>e</span> aaddddiittiioonnaall BBUUAA ttoo tthhee eexxtteenntt ooff 1155%% ooff eexxiissttiinngg
development<span style='color:red'>,</span> on account of other laws or regulations that are<span style='color:red'> </span>binding.
BBUUAA oorr 1100 ssqq.. mm<span style='color:red'> </span>ppeerr<span style='color:red'> </span>tteenneemmeenntt wwhhiicchheevveerr iiss mmoorre<span style='color:red'>e</span> sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e </span>ppeerrmmiissssiibblle<span style='color:red'>e</span>
Owners of<span style='color:red'> land</span> parcels having plot area lesser than 2 ha may come together
wwiitthhoouutt pprre<span style='color:red'>em</span>miiuumm..
to create contiguous<span style='color:red'> land</span> parcels of 2 ha or more & submit proposal<span style='color:red'> for</span>
PPrroovviiddeedd ffuurrtthheerr tthhaatt iiff tthhee eexxiissttiinngg<span style='color:red'> a</span>auutthhoorriizzeedd BBUUAA aanndd iinncceennttiivve<span style='color:red'>e</span>
development under this Regulation along with proper access as per these
tthheerre<span style='color:red'>e</span>oonn<span style='color:red'> a</span>ass ssttaatteedd<span style='color:red'> a</span>abboovve<span style='color:red'>e</span> iiss lle<span style='color:red'>e</span>ssss tthhaann tthhee<span style='color:red'> </span>ppeerrmmiissssiibblle<span style='color:red'>e</span> FFSSII<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr
Regulations. However<span style='color:red'>,</span> the Municipal Commissioner shall sanction the
rreegguullaattiioonn 3300((AA))((11)),<span style='color:red'>,</span> tthheenn ssoocciieettyy mmaayy<span style='color:red'> a</span>avvaaiill ‘‘AAddddiittiioonnaall FFSSII oonn<span style='color:red'> </span>ppaayymmeenntt
proposal with prior approval of Govt.
ooff pprre<span style='color:red'>em</span>miiuumm//TTDDRR’’ uupp ttoo<span style='color:red'> </span>lliimmiitt ooff<span style='color:red'> </span>ppeerrmmiissssiibblle<span style='color:red'>e</span> FFSSII..
(B) Planning Considerations /Submission of Proposal
22)) IIff ssttaaiirrccaasse<span style='color:red'>e,, </span>lliifftt &&<span style='color:red'> </span>lliifftt lloobbbbyy<span style='color:red'> a</span>arre<span style='color:red'>e</span>aass<span style='color:red'> a</span>arre<span style='color:red'>e </span>ccllaaiimmeedd ffrreeee ooff FFSSII<span style='color:red'> </span>bbyy cchhaarrggiinngg
The proposal shall be submitted by the Owner<span style='color:red'>,</span> containing the demand
pprre<span style='color:red'>em</span>miiuumm<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthheenn pprreevvaaiilliinngg RReegguullaattiioonn<span style='color:red'>,,</span> tthheenn ssuucchh<span style='color:red'> a</span>arre<span style='color:red'>e</span>aass ttoo tthhaatt
assessment<span style='color:red'> for</span> infrastructure such as roads<span style='color:red'>,</span> water supply<span style='color:red'>,</span> sewerage and
eexxtteenntt oonnllyy wwiillll<span style='color:red'> </span>bbe<span style='color:red'>e</span> ggrraanntteedd ffrreeee ooff FFSSII wwiitthhoouutt cchhaarrggiinngg pprre<span style='color:red'>em</span>miiuumm.. IIff
storm water drains along with clearly earmarking the area<span style='color:red'> for</span> Public Open
ssttaaiirrccaasse<span style='color:red'>e,, </span>lliifftt &&<span style='color:red'> </span>lliifftt lloobbbbyy<span style='color:red'> a</span>arre<span style='color:red'>e</span>aass<span style='color:red'> a</span>arre<span style='color:red'>e </span>ccoouunntteedd iinn FFSSII iinn eeaarrlliieerr
Spaces (POS), Affordable Housing (AH), Other Amenities (OA), (viz. Education<span style='color:red'>,</span>
ddeevveellooppmmeenntt<span style='color:red'>,,</span> tthheenn iinncceennttiivve<span style='color:red'>e</span> aaddddiittiioonnaall FFSSII<span style='color:red'> a</span>ass ssttaatteedd iinn SSrr.. NNoo.. 11 sshhaallll
Health & Social Amenities) and area<span style='color:red'> for</span> other development i.<span style='color:red'>e</span>. owners share<span style='color:red'>
a</span>allssoo<span style='color:red'> </span>bbe<span style='color:red'>e </span>ggiivveenn oonn ssuucchh<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa && ssuucchh<span style='color:red'> a</span>arre<span style='color:red'>e</span>aass mmaayy<span style='color:red'> </span>bbe<span style='color:red'>e a</span>avvaaiilleedd ffrreeee ooff FFSSII<span style='color:red'> </span>bbyy
of<span style='color:red'> land</span>. If OA/POS/AH to be handed over to MCGM<span style='color:red'> is</span> not abutting the
cchhaarrggiinngg pprre<span style='color:red'>em</span>miiuumm<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthheesse<span style='color:red'>e</span> RReegguullaattiioonnss..
municipal road<span style='color:red'>,</span> the same shall be provided with uninterrupted access as per
table no 7 of Regulation No 23(1). The area of the<span style='color:red'> land</span> after<span style='color:red'> de</span>duction of the
33)) TThhiiss RReegguullaattiioonn sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e a</span>apppplliiccaabblle<span style='color:red'>e</span> oonnllyy wwhheenn eexxiissttiinngg<span style='color:red'> </span>mme<span style='color:red'>em</span>mbbeerrss ooff tthhee area covered under road/uninterrupted access proposed as above<span style='color:red'>,</span> shall be
ssoocciieettiieess<span style='color:red'> a</span>arre<span style='color:red'>e</span> pprrooppoosseedd ttoo<span style='color:red'> </span>bbe<span style='color:red'>e</span> rre<span style='color:red'>e</span>--aaccccoommmmooddaatteedd..<span style='color:red'> </span>apportioned among Owner’s Share<span style='color:red'>,</span> AH<span style='color:red'>,</span> POS<span style='color:red'>,</span> and OA as detailed below.
44)) TThhiiss rreegguullaattiioonn wwiillll<span style='color:red'> </span>bbe<span style='color:red'>e a</span>apppplliiccaabblle<span style='color:red'>e</span> ffoorr rreeddeevveellooppmmeenntt ooff eexxiissttiinngg These roads/uninterrupted access shall be handed over to MCGM and will be<span style='color:red'>
a</span>auutthhoorriizzeedd bbuuiillddiinnggss wwhhiicchh<span style='color:red'> a</span>arre<span style='color:red'>e</span> ooff tthhiirrttyy yye<span style='color:red'>e</span>aarrss ooff<span style='color:red'> a</span>agge<span style='color:red'>e</span> oorr mmoorre<span style='color:red'>e</span>.. eligible<span style='color:red'> for</span> additional BUA equal to area of<span style='color:red'> land</span> surrendered/transferred over
55)) TThhiiss rreegguullaattiioonn sshhaallll nnoott<span style='color:red'> </span>bbe<span style='color:red'>e a</span>apppplliiccaabblle<span style='color:red'>e</span> iinn rreessppe<span style='color:red'>e</span>cctt ooff rreeddeevveellooppmmeenntt
and above FSI as stipulated below in Sr. No. D.
pprrooppoossaall ttoo<span style='color:red'> </span>bbe<span style='color:red'>e</span>//bbeeiinngg pprroocceesssseedd uunnddeerr RReegguullaattiioonn NNoo.. 3333((55)),<span style='color:red'>,</span> 3333((77)),<span style='color:red'>,</span>
Sr. Plot Affordable Public Other Area<span style='color:red'> for</span>
3333((88)),<span style='color:red'>,</span> 3333((99)),<span style='color:red'>,</span> 3333((99))((AA)),<span style='color:red'>,</span> 3333((99))((BB)),<span style='color:red'>,</span> 3333((1100)),<span style='color:red'>,</span> 3333((1100)) ((AA)),<span style='color:red'>,</span> 3333((2200)) ((AA)),<span style='color:red'>,</span> 3333((2211))..
No Area Housing Open Amenities Other
EExxppllaannaattiioonn:: --AAgge<span style='color:red'>e</span> ooff<span style='color:red'> a</span>a bbuuiillddiinngg sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e a</span>ass oonn tthhee 11sstt ooff JJaannuuaarryy ooff tthhee<span style='color:red'> </span>. Spaces Development
yye<span style='color:red'>e</span>aarr iinn wwhhiicchh<span style='color:red'> a</span>a<span style='color:red'> </span>ccoommpplleette<span style='color:red'>e</span> rreeddeevveellooppmmeenntt pprrooppoossaall iiss ssuubbmmiitttteedd ttoo tthhee 1 Not less 30% 15 % 14% 41%
CCoommmmiissssiioonneerr aanndd sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> ccaallccuullaatteedd ffrroomm tthhee ddaatte<span style='color:red'>e</span> ooff<span style='color:red'> </span>OOccccuuppaattiioonn than 2 ha
CCeerrttiiffiiccaatte<span style='color:red'>e</span> oorr<span style='color:red'> a</span>alltteerrnnaatteellyy<span style='color:red'>,,</span> ffrroomm tthhee<span style='color:red'> </span>ffiirrsstt ddaatte<span style='color:red'>e</span> ooff<span style='color:red'> a</span>asssse<span style='color:red'>e</span>ssssmmeenntt<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthhee & up to 4
ha
pprrooppeerrttyy<span style='color:red'> </span>ttaaxx rre<span style='color:red'>e</span>ccoorrdd iinn rreessppe<span style='color:red'>e</span>cctt ooff ssuucchh bbuuiillddiinngg oorr bbuuiillddiinngg oonn wwhhiicchh
2 More 30% 15 % 12% 43%<span style='color:red'>
</span>bbaallaanncce<span style='color:red'>e</span>//aaddmmiissssiibblle<span style='color:red'>e</span> FFSSII hhaass<span style='color:red'> </span>bbe<span style='color:red'>e</span>eenn<span style='color:red'> </span>ccoonnssuummeedd<span style='color:red'> </span>bbyy wwaayy ooff vveerrttiiccaall
than 4 ha
eexxtteennssiioonn<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthheenn pprreevvaaiilliinngg RReegguullaattiioonn<span style='color:red'>,, a</span>avvaaiillaabblle<span style='color:red'>e</span> wwiitthh tthhee MMCCGGMM..<span style='color:red'>
</span>66)) TThhiiss iinncceennttiivve<span style='color:red'>e</span> aaddddiittiioonnaall BBUUAA sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> iinndde<span style='color:red'>e</span>ppeennddeenntt ooff aaddddiittiioonnaall BBUUAA<span style='color:red'>
a</span>ass<span style='color:red'> </span>ppeerrmmiissssiibblle<span style='color:red'>e</span> uunnddeerr RReegguullaattiioonn NNoo.. 1144((AA)),<span style='color:red'>,</span> 1155<span style='color:red'>,,</span> 1166 aanndd 1177<span style='color:red'>,,</span> iiff aannyy.. The Owner shall submit his proposal as per above to the Commissioner
77)) FFuunnggiibblle<span style='color:red'>e </span>ccoommppeennssaattoorryy<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa aaddmmiissssiibblle<span style='color:red'>e</span> uunnddeerr RReegguullaattiioonn NNoo.. 3311((33)) oonn MCGM. While making such submission<span style='color:red'>,</span> he will take care of the following:
tthhee eexxiissttiinngg<span style='color:red'> a</span>auutthhoorriisseedd BBUUAA sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> wwiitthhoouutt cchhaarrggiinngg pprre<span style='color:red'>em</span>miiuumm aanndd
(a) He shall distinctly mark lands<span style='color:red'> for</span> AH<span style='color:red'>,</span> POS<span style='color:red'>,</span> OA and Owner’s share in the layout.
oovveerr tthhee iinncceennttiivve<span style='color:red'>e</span> aaddddiittiioonnaall BBUUAA<span style='color:red'> </span>bbyy cchhaarrggiinngg ooff pprre<span style='color:red'>em</span>miiuumm..
Further earmarking of lands<span style='color:red'> for</span> other amenities like education<span style='color:red'>,</span> health and social<span style='color:red'>
</span>88)) IIff tteennaanntteedd bbuuiillddiinngg//ss aanndd bbuuiillddiinngg//ss ooff<span style='color:red'> </span>ccoo--ooppeerraattiivve<span style='color:red'>e</span> hhoouussiinngg
amenities cited above shall be done by the Commissioner taking the amenity
ssoocciieettyy//nnoonn-- tteennaanntteedd bbuuiillddiinngg//ss<span style='color:red'> </span>ccoo--<span style='color:red'>e</span>exxiisstt oonn tthhee pplloott uunnddeerr
standards prescribed as minimum.
ddeevveellooppmmeenntt<span style='color:red'>,,</span> tthheenn pprrooppoorrttiioonnaatte<span style='color:red'>e </span>llaanndd<span style='color:red'> </span>ccoommppoonneenntt<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthhee eexxiissttiinngg
(b) Advance possession of all lands other than the Owner’s Share as detailed in<span style='color:red'>
a</span>auutthhoorriisseedd BBUUAA ooff eexxiissttiinngg tteennaanntteedd bbuuiillddiinngg oonn tthhee pplloott sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span>
the table above shall be handed over to MCGM at the time of approval of
118800 DCPR 2034 181 DCPR 2034<span style='color:red'>
</span>

In [69]:
# Hybrid search results
display(Markdown("\n**Hybrid Search Results:**"))
formatted_results = doc_text_formatting(ef, query, hybrid_results)
for result in formatted_results:
    display(Markdown(result))


**Hybrid Search Results:**

Original Notifications shall
be referred<span style='color:red'> for</span> perspicuity
vest<span style='color:red'>,</span> by provision in<span style='color:red'> a de</span>ed of<span style='color:red'> conveyance,</span> in all the owners on
account of whose cumulative holdings the LOS<span style='color:red'> is</span> required to be
kept as LOS in the layout or sub-division/amalgamation/plot of the<span style='color:red'>
land</span>.
(b) The proposal<span style='color:red'> for</span> construction of such structure should come as<span style='color:red'> a</span>
proposal from the owner/owners/society/soci<span style='color:red'>e</span>ties or federation of
societies shall be meant<span style='color:red'> for</span> the beneficial use of the owner/owners/
members of such<span style='color:red'> society</span>/soci<span style='color:red'>e</span>ties/federation of societies.
(c) Such structures shall not be used<span style='color:red'> for</span> any other purpose<span style='color:red'>,</span> except<span style='color:red'> for</span>
recreational activities.
(d) The remaining area of the LOS shall be kept open to sky and
accessible to all members as<span style='color:red'> a</span> place of recreation.
(<span style='color:red'>e</span>) The owner/owners/or<span style='color:red'> society</span> or societies or federation of the
societies shall submit to the Commissioner<span style='color:red'> a</span> registered undertaking
agreeing to the conditions in (a) to (d) above.
(f) LOS in<span style='color:red'> a</span> private layout shall be<span style='color:red'> for</span> the exclusive use of the residents
of such private layout only and shall not be subjected to<span style='color:red'> </span>acquisition by
MCGM/Appropriate Authority. Further in such cases area of
existing Recreational Open Space shall have to be maintained by
residents of such private layout.
(h) Structures/Uses permitted in layout open spaces:
“Construction of Solid Waste Management System as per the
National Building Code of India<span style='color:red'>,</span> Part 9 Plumbing Services<span style='color:red'>,</span> Section 1-
Water Supply<span style='color:red'>,</span> Drainage & Sanitation (including Solid Waste
Management) paragraph 6 /bio-degradable waste treatment plant<span style='color:red'>,</span>
in the layout LOS<span style='color:red'>,</span> having area 2000 Sq.mt. & above within 10% of the
LOS area.”
(2) LOS in industrial plots/layout of industrial plots in any industrial plot
admeasuring 1000 sq. m or more in area<span style='color:red'>,</span> 15 per cent of the total area
shall be provided as LOS subject to:
(i.) Such LOS shall have proper means of access and shall be so located
that it<span style='color:red'> can</span> be conveniently utilized by the persons working in the
industry;
(ii.) Such LOS shall be kept permanently open to sky and accessible to
all the owners and occupants and trees shall be grown therein at
the rate of 5 trees<span style='color:red'> for</span> every 100 sq. m of the said open space or
114 DCPR 2034<span style='color:red'>
</span>

Original Notifications shall
be referred<span style='color:red'> for</span> perspicuity
3.11 Notwithstanding anything contained in this regulation<span style='color:red'>,</span>
rehabilitation project of<span style='color:red'> a</span> slum located on<span style='color:red'> land</span> belonging to
public authority and needed<span style='color:red'> for a</span> vital public purpose and
where eligible slum dwellers which cannot be accommodated
in the in-situ SRS of<span style='color:red'> land</span> under non-buildable reservations<span style='color:red'>, is</span>
taken up on an unencumbered plot<span style='color:red'>,</span> TDR as per regulation 32
table 12(A)<span style='color:red'> for</span> the area of the<span style='color:red'> land</span> spared<span style='color:red'> for</span> this purpose shall
be sanctioned to the owner of the said unencumbered plot and
the TDR in lieu of cost of construction of BUA as per sub-
regulation 4.2 of regulation 32(A) shall be permissible.. For the
purpose of this Regulation<span style='color:red'>,</span> the BUA shall be as per claus<span style='color:red'>e</span> 3.2 of
this Regulation. No sale component shall be permissible.
Provided that the State Govt. or Public authority or<span style='color:red'> a</span> Govt.
Company as defined in Sec. 617 of the Companies Act 1956 and<span style='color:red'>
</span>owned and controlled by the State Govt. (herein after referred
as the Agency) may undertake Slum Rehabilitation Project on its
own<span style='color:red'> land</span> and be eligible<span style='color:red'> for</span> the benefits under this Regulation
subject to following conditions:
1) The Rehabilitation Project<span style='color:red'> is</span> approved by the SRA.
2) The tenements so constructed in execution of the Project are offered
to slum dwellers located on<span style='color:red'> land</span> belonging to Govt. or Public Authority
and needed<span style='color:red'> for</span> vital public purpose and within 270 days from the date
of issue of LOI the Agency shall identify the slum dwellers.
3) If the Agency fails to identify the slum dwellers needed to be shifted<span style='color:red'>
for a</span> vital public purpose<span style='color:red'>,</span> as above<span style='color:red'>,</span> then the tenements so constructed
shall be offered.<span style='color:red'>
a</span>) to the slum dwellers located on<span style='color:red'> land</span> belonging to Government or Public
Authority within<span style='color:red'> a</span> distance of 2 km. from the<span style='color:red'> land</span> on which the Project<span style='color:red'>
is</span> undertaken<span style='color:red'>,</span> or
b) to the slum dwellers located anywhere in Greater Mumbai on lands
belonging to Govt. or Public Authority<span style='color:red'>,</span> or
4) Further provided that in all above cases the relocation of slum dwellers
in any case will be undertaken not with reference to individuals but
reference to<span style='color:red'> a</span>ssembly of slum dwellers<span style='color:red'> for</span> the purpose of releasing
the plot of<span style='color:red'> land</span> wholly from slums and not only the patches of<span style='color:red'> land</span>.
Provided that notwithstanding anything mentioned above<span style='color:red'>,</span> project
affected persons due to any vital Public Projects undertaken by
MMRDA including PAP’s under Mumbai Urban Transport Project
(MUTP) being resettled as per the provisions contained in Government
Resolution<span style='color:red'>,</span> Housing and Special Assist<span style='color:red'>ance</span> Department<span style='color:red'>,</span> by order no.
700/CR 31/slum-2 dated 12/12/2000 and certified by the Project
Director<span style='color:red'>,</span> MUTP will also be eligible<span style='color:red'> for</span> redevelopment scheme under
this Regulation<span style='color:red'>,</span> as amended from time to time.
228 DCPR 2034<span style='color:red'>
</span>

Original Notifications shall Original Notifications shall
be referred<span style='color:red'> for</span> perspicuity be referred<span style='color:red'> for</span> perspicuity
iinn tthhee<span style='color:red'> </span>ssaamme<span style='color:red'>e</span> sseeccttoorr oorr ootthheerr sseeccttoorrss wwiitthhiinn tthhee jjuurriissddiiccttiioonn ooff DDRRPP..
1.11 ‘Premium<span style='color:red'> for</span> ownership and terms of lease-. That part of
11..44 PPaavveemmeenntt ddwweelllleerrss aanndd hhuuttmmeenntt ddwweelllleerrss iinn tthhee sslluumm oonn tthhee<span style='color:red'> </span>llaanndd Government/MCGM/MHADA<span style='color:red'> land</span> on which the rehabilitation
rre<span style='color:red'>e</span>qquuiirreedd ffoorr vviittaall uurrggeenntt<span style='color:red'> </span>ppuubblliicc uuttiilliittyy//ppuurrppoossee oorr oonn tthhee hhaazzaarrddoouuss component of the SRS will be constructed shall be leased to the Co-
operativ<span style='color:red'>e</span> Housing Society of the slum-dwellers<span style='color:red'> for</span> 30 years. Annual
llooccaattiioonn oorr<span style='color:red'> a</span>affffe<span style='color:red'>e</span>cctteedd<span style='color:red'> </span>bbyy<span style='color:red'> </span>DDPP pprrooppoossaallss sshhaallll nnoott<span style='color:red'> </span>bbe<span style='color:red'>e</span> rreehhaabbiilliittaatteedd iinn--ssiittuu
bbuutt iinn ootthheerr<span style='color:red'> a</span>avvaaiillaabblle<span style='color:red'>e</span> pplloottss wwiitthhiinn jjuurriissddiiccttiioonn ooff DDRRPP.. lease rent of Rs. 1001<span style='color:red'> for</span> 4000 per sq. m. of<span style='color:red'> land</span> or part thereof and
lease shall be renewable<span style='color:red'> for a</span> further period of 30 years at<span style='color:red'> a</span> time.
11..55 AA<span style='color:red'> </span>cceerrttiiffiieedd eexxttrraacctt ooff tthhee rreelleevvaanntt eelle<span style='color:red'>e</span>ccttoorraall<span style='color:red'> </span>rroollll sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e </span>ccoonnssiiddeerreedd Simultaneously<span style='color:red'>, land</span> under free sale component shall be leased
aadde<span style='color:red'>e</span>qquuaatte<span style='color:red'>e</span> eevviiddeenncce<span style='color:red'>e</span> ttoo eessttaabblliisshh tthhee eelliiggiibbiilliittyy ooff<span style='color:red'> a</span>a<span style='color:red'> </span>ppeerrssoonn pprroovviiddeedd hhee directly to the Society/Association of the purchasers of the tenement
iiss<span style='color:red'> </span>ffoouunndd rreessiiddiinngg iinn tthhee ssttrruuccttuurre<span style='color:red'>e</span>.. TThhiiss iiss ttoo<span style='color:red'> a</span>avvooiidd tthhee<span style='color:red'> </span>ppoossssiibbiilliittyy ooff under free sale component. Pending the formation of the<span style='color:red'>
</span>ppeerrssoonnss wwhhoo hhaavve<span style='color:red'>e</span> lleefftt tthhee ssttrruuccttuurre<span style='color:red'>e </span>ccoommiinngg<span style='color:red'> </span>bbaacckk ttoo<span style='color:red'> </span>ccllaaiimm ffrreeee Society/Association of the purchasers in the free sale component with<span style='color:red'>
</span>tteenneemmeenntt uunnddeerr tthhee sscchheemme<span style='color:red'>e</span> eevveenn tthhoouugghh tthheeyy hhaavve<span style='color:red'>e</span> iinn tthhee nnoorrmmaall<span style='color:red'> a</span> provision<span style='color:red'> for</span> further renewal<span style='color:red'> for a</span> period of 30 years at<span style='color:red'> a</span> time. The<span style='color:red'>
</span>ccoouurrssee lleefftt tthhee sslluumm aanndd ggoonne<span style='color:red'>e a</span>awwaayy iinnttoo<span style='color:red'> a</span>a pprrooppeerr nnoonn--sslluumm<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa oorr oouutt lease rent<span style='color:red'> for</span> the free sale component shall be fixed by SRA.
ooff DDRRPP AArre<span style='color:red'>e</span>aa.. IIff tthhee hhuuttmmeenntt ddwweelllleerrss<span style='color:red'> a</span>arre<span style='color:red'>e </span>ffoouunndd rreessiiddeenntt iinn tthhee
ssttrruuccttuurre<span style='color:red'>e,,</span> bbuutt tthhee<span style='color:red'> </span>nnaammeess<span style='color:red'> a</span>arre<span style='color:red'>e</span> oonn tthhee eelle<span style='color:red'>e</span>ccttoorraall<span style='color:red'> </span>rroollll oonn oorr pprriioorr ttoo 11sstt JJaann In addition to above<span style='color:red'>,</span> the Developer/Co-op. Housing Society shall pay
22000000<span style='color:red'>,, a</span>att aannootthheerr sslluumm//ppaavveemmeenntt ssiitte<span style='color:red'>e</span> iinn BBrriihhaann MMuummbbaaii<span style='color:red'>,,</span> tthheeyy sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> premium at the rate of 25% of ASR in respect of SRS proposed to be<span style='color:red'>
</span>ccoonnssiiddeerreedd eelliiggiibblle<span style='color:red'>e</span> bbuutt oonnllyy<span style='color:red'> a</span>att tthhee ppllaacce<span style='color:red'>e</span> ooff pprreesseenntt rreessiiddeenncce<span style='color:red'>e</span>.. IInn ccaasse<span style='color:red'>e</span> undertaken on lands<span style='color:red'> </span>owned by Government<span style='color:red'>,</span> Semi-Government
ooff ddoouubbtt oorr ddiissppuutte<span style='color:red'>e,,</span> tthhee dde<span style='color:red'>e</span>cciissiioonn ooff tthhee CCoommppeetteenntt AAuutthhoorriittyy ttoo<span style='color:red'> </span>bbe<span style='color:red'>e</span> undertakings and Local Bodies and premium shall go to<span style='color:red'> land</span> owing<span style='color:red'>
a</span>appppooiinntteedd<span style='color:red'> </span>bbyy tthhee GGoovvtt.. iinn HHoouussiinngg DDeeppaarrttmmeenntt sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e </span>ffiinnaall aanndd authority such as MHADA<span style='color:red'>,</span> MCGM<span style='color:red'>,</span> MMRDA as the case may be. The<span style='color:red'>
</span>bbiinnddiinngg oonn<span style='color:red'> a</span>allll tthhee ppaarrttiieess<span style='color:red'> </span>ccoonncceerrnneedd.. premium installment so recovered shall be remitted to concerned<span style='color:red'> land</span>
owing authority within 30 days from the date of recovery.
11..66 AAnn iinnddiivviidduuaall<span style='color:red'> a</span>aggrreeeemmeenntt sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> eenntteerreedd iinnttoo<span style='color:red'> </span>bbyy tthhee ddeevveellooppeerr ssoo<span style='color:red'>
a</span>appppooiinntteedd uunnddeerr DDRRPP<span style='color:red'> </span>bbyy DDRRPP ((SSRRAA)) wwiitthh tthhee eelliiggiibblle<span style='color:red'>e</span> hhuuttmmeenntt ddwweelllleerrss
In the case of Govt.<span style='color:red'> land,</span> the premium shall be deposited in Nivara
ooff eeaacchh ssttrruuccttuurre<span style='color:red'>e</span> iinn tthhee sslluumm//ppaavveemmeenntt.. Nidhi.
The amount of premium shall be recovered in installment as may be
11..77 TThhee iinnddiivviidduuaall<span style='color:red'> a</span>aggrreeeemmeenntt eenntteerreedd iinnttoo<span style='color:red'> </span>bbeettwweeeenn hhuuttmmeenntt ddwweelllleerrss aanndd
prescribed by Govt. from time to time. Land owning authority such as
tthhee<span style='color:red'> </span>llaanndd--oowwnniinngg<span style='color:red'> a</span>auutthhoorriittyy//SSRRAA//ddeevveellooppeerr sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> iinn tthhee jjooiinntt<span style='color:red'> </span>nnaammeess
MCGM<span style='color:red'>,</span> MMRDA<span style='color:red'>,</span> MHADA shall not recover<span style='color:red'> land</span> premium in any other
ooff pprraammuukkhh hhuuttmmeenntt ddwweelllleerr aanndd ssppoouusse<span style='color:red'>e</span> ffoorr eevveerryy ssttrruuccttuurre<span style='color:red'>e</span>..
form. Proposals<span style='color:red'> for</span> SRS on<span style='color:red'> land </span>owned by Central Govt. shall be
accepted only after NOC<span style='color:red'> for</span> the scheme<span style='color:red'> is</span> obtained from the
11..88 HHuuttmmeennttss hhaavviinngg<span style='color:red'> a</span>a pphhyyssiiccaallllyy hhaannddiiccaappppeedd<span style='color:red'> </span>ppeerrssoonn oorr<span style='color:red'> </span>ffe<span style='color:red'>em</span>maallee--hheeaaddeedd
concerned Central Govt. Department.
hhoouusseehhoollddss sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e </span>ggiivveenn<span style='color:red'> </span>ffiirrsstt pprreeffeerreenncce<span style='color:red'>e</span> iinn<span style='color:red'> a</span>alllloottmmeenntt ooff<span style='color:red'> </span>tteenneemmeennttss
ttoo tthhee ootthheerr hhuuttmmeenntt--ddwweelllleerrss.. TThheerre<span style='color:red'>e</span>aafftteerr lloottss sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e </span>ddrraawwnn ffoorr 1.12 Automatic cancellation of Vacant Land Tenure – If any<span style='color:red'> land</span> or part of<span style='color:red'>
a</span>alllloottmmeenntt ooff<span style='color:red'> </span>tteenneemmeennttss ffrroomm tthhee rre<span style='color:red'>em</span>maaiinniinngg<span style='color:red'> </span>tteenneemmeennttss ttoo tthhee ootthheerr any<span style='color:red'> land</span> on which slum<span style='color:red'> is</span> located<span style='color:red'> is</span> under vacant<span style='color:red'> land</span> tenure the said
tenure/lease created by MCGM or Municipal Commissioner shall stand
hhuuttmmeenntt--ddwweelllleerrss..
automatically terminated as soon as DRP<span style='color:red'>,</span> which<span style='color:red'> is a</span> public purpose<span style='color:red'>,</span>
on such<span style='color:red'> land is</span> prepared and submitted<span style='color:red'> for</span> approval to the DRP cell of
11..99 TTrraannssffeerr ooff PPhhoottoo<span style='color:red'> </span>ppaasssseess –– SSiinncce<span style='color:red'>e</span> oonnllyy tthhee<span style='color:red'> a</span>accttuuaall ooccccuuppaanntt<span style='color:red'> a</span>att pprreesseenntt
SRA. Any arrears of dues to be collected by MCGM shall not be linked
wwiillll<span style='color:red'> </span>bbe<span style='color:red'>e</span> eelliiggiibblle<span style='color:red'>e</span> ffoorr rreeddeevveellooppmmeenntt<span style='color:red'>,,</span> tthheerre<span style='color:red'>e</span> sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> nnoo<span style='color:red'> </span>nneeeedd ttoo rreegguullaarriizze<span style='color:red'>e</span>
to the issue of any certificate or NOC relating to the DRP.
tthhee<span style='color:red'> </span>ttrraannssffeerrss ooff pphhoottoo<span style='color:red'> </span>ppaasssseess tthhaatt hhaavve<span style='color:red'>e</span> ooccccuurrrreedd ssoo<span style='color:red'> </span>ffaarr..
1.13 Recovery of<span style='color:red'> </span>pending dues such as assessment<span style='color:red'>,</span> compensation<span style='color:red'>,</span>
11..1100 AAnnyy<span style='color:red'> </span>ppeerrssoonn wwhhoossee<span style='color:red'> </span>nnaamme<span style='color:red'>e</span> iiss eennrroolllleedd iinn<span style='color:red'> a</span>a nnoonn--sslluumm<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa iinn BBrriihhaann
occupational charges<span style='color:red'>,</span> non-agricultural tax/dues etc.<span style='color:red'> </span>pending with
MMuummbbaaii bbuutt hhaass<span style='color:red'> </span>ppuurrcchhaasseedd<span style='color:red'> a</span>a hhuuttmmeenntt iinn DDRRPP<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa aanndd tthheerreeffoorre<span style='color:red'>e</span> ggoott
public authorities such as State Govt.<span style='color:red'>,</span> MHADA<span style='color:red'>,</span> and/or MCGM shall be
hhiiss<span style='color:red'> </span>nnaamme<span style='color:red'>e a</span>allssoo iinncclluuddeedd iinn eelle<span style='color:red'>e</span>ccttoorraall<span style='color:red'> </span>rroollll ffoorr tthhee sslluumm<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa<span style='color:red'>,,</span> ii..ee.. hhee hhaass hhiiss<span style='color:red'>
de</span>alt with separately and not be linked to grant of approval or building<span style='color:red'>
</span>nnaamme<span style='color:red'>e</span> iinn tthhee eelle<span style='color:red'>e</span>ccttoorraall<span style='color:red'> </span>rroollll<span style='color:red'> a</span>att ttwwoo ppllaacceess<span style='color:red'>,,</span> hhee sshhaallll nnoott<span style='color:red'> </span>bbe<span style='color:red'>e</span> eelliiggiibblle<span style='color:red'>e</span> ffoorr tthhee
permission to the DRP.
sscchheemme<span style='color:red'>e</span>..
224444 DCPR 2034 245 DCPR 2034<span style='color:red'>
</span>

Original Notifications shall
be referred<span style='color:red'> for</span> perspicuity
Provided that when the development<span style='color:red'> is</span> composit<span style='color:red'>e</span> and where it<span style='color:red'> is</span> not
possible to hand over part of<span style='color:red'> land</span> to MCGM in such cases entire
area shall be<span style='color:red'> deem</span>ed to be marked as existing amenity with the +
sign indicating other uses on the said plot.
20) General conditions to allow development under above regulations:
i) If the area of reservation<span style='color:red'> is</span> not adequat<span style='color:red'>e</span> to construct independent
building as mentioned above OR When it<span style='color:red'> is</span> not possible to handover
individual plot along with public amenity<span style='color:red'>,</span> then in such cases Municipal
Commissioner may allow composit<span style='color:red'>e</span> building on said<span style='color:red'> land</span> subject to
condition that the built-up area mentioned as above may be allowed
to be handed over to the Planning Authority or Appropriate Authority<span style='color:red'>,</span>
as the case may be<span style='color:red'>,</span> preferably on ground floor and subject to premium
at the rate of 35% and 40% of ASR of the develop<span style='color:red'> land for</span> the zonal
(basic) FSI (In case of suburbs where FSI<span style='color:red'> is</span> 1 it shall be as per ASR of the
developed<span style='color:red'> land</span> and in case of City it shall be 1.33 times of ASR of the
developed<span style='color:red'> land</span>)<span style='color:red'> for</span> AH/R&R and<span style='color:red'> for</span> other reservations respectively or
as may be decided by Government from time to time. If ground floor<span style='color:red'>
is</span> utilised<span style='color:red'> for</span> parking<span style='color:red'>,</span> then the built-up area mentioned above may be
handed over on stilt/first floor with separate entry & exit from public
street. In such cases<span style='color:red'>,</span> built- up area along with proportionate undivided
share of<span style='color:red'> land</span> shall be handed over to the Planning Authority or
Appropriate Authority<span style='color:red'>,</span> as the case may be. In such cases no
compensation of proportionate undivided<span style='color:red'> land</span> share shall be
permissible to the<span style='color:red'> land</span> owner / developer.
ii) It shall be obligatory on Municipal Commissioner to make registered
agreement with the developer /owner at the time of granting the
development permission subject to terms and conditions as it<span style='color:red'> deem</span>
fit. Occupancy Certificate shall be issued only after compliance of all
terms & conditions and getting possession of the constructed
amenity.
iii) The above permissions<span style='color:red'> for</span> development of reservations shall be
granted by the Municipal Commissioner as per the norms mentioned
in these regulations.
iv) The area / built-up area to be handed over to the Planning Authority
under these Regulations shall be earmarked on the sanctioned
building plan clearly mentioning the same<span style='color:red'>,</span> and registered agreement
to that effect shall be executed. After completion of construction<span style='color:red'>,</span> the
said amenity shall be handed over by executing the<span style='color:red'> de</span>ed of transfer
88 DCPR 2034<span style='color:red'>
</span>

Original Notifications shall Original Notifications shall
be referred<span style='color:red'> for</span> perspicuity be referred<span style='color:red'> for</span> perspicuity
((99)) IInn ccaasse<span style='color:red'>e</span> ooff<span style='color:red'> </span>ccoonnvveerrssiioonn//rreevviissiioonn ooff LLOOII<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthhiiss rreegguullaattiioonn aanndd wwhheerre<span style='color:red'>e</span>
pplloott iiss rreesseerrvveedd ffoorr ddiiffffeerreenntt//ssaamme<span style='color:red'>e </span>ppuubblliicc<span style='color:red'> </span>ppuurrppoossee ffoorr eeqquuaall oorr mmoorre<span style='color:red'>e
a</span>arre<span style='color:red'>e</span>aa ooff rreesseerrvvaattiioonn iinn<span style='color:red'> </span>DDPP 22003344 tthhaann<span style='color:red'> </span>DDPP 11999911<span style='color:red'>,,</span> tthheenn ddeevveellooppmmeenntt sshhaallll of MCGM/Appropriate Authority. Proforma of possession receipt shall<span style='color:red'>
</span>bbe<span style='color:red'>e a</span>ass<span style='color:red'> </span>ppeerr tthhiiss rreegguullaattiioonn.. TThhee<span style='color:red'> </span>DDPP RRooaadd sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> rreecckkoonneedd wwiitthh<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr be as per Appendix V.<span style='color:red'>
</span>DDPP 22003344..
(6) The area of built-up amenity shall be counted in FSI initially and after
((bb)) SSlluummss iinn IInndduussttrriiaall ZZoonne<span style='color:red'>e</span> ((II))
handing over of said built up amenity the area of built-up amenity shall
((11)) SSlluummss iinn IInndduussttrriiaall ZZoonne<span style='color:red'>e</span> ((II)) sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e a</span>alllloowweedd ttoo<span style='color:red'> </span>bbe<span style='color:red'>e</span> rreeddeevveellooppeedd iinn--ssiittuu be allowed free of FSI. Commencement Certificate in respect of BUA in
wwiitthhoouutt ggooiinngg tthhrroouugghh tthhee pprroocce<span style='color:red'>e</span>ssss ooff cchhaannggee ooff<span style='color:red'> </span>zzoonne<span style='color:red'>e</span>.. IInn tthhee ffrreeee--ssaalle<span style='color:red'>e</span> lieu of the built-up amenity handed over to MCGM<span style='color:red'> can</span> be granted only<span style='color:red'>
</span>ccoommppoonneenntt iinn aannyy<span style='color:red'> </span>zzoonne<span style='color:red'>e,,</span> iinn aaddddiittiioonn ttoo rreessiiddeennttiiaall uusseess<span style='color:red'>,, a</span>allll tthhee uusseess after handing over of such built up amenity or before availing zonal<span style='color:red'>
</span>ppeerrmmiitttteedd ffoorr tthhee oorriiggiinnaall<span style='color:red'> </span>zzoonne<span style='color:red'>e</span> sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e </span>ppeerrmmiitttteedd.. FFoorr iinndduussttrriiaall uusseess<span style='color:red'>,,</span> (basic) FSI beyond 75% of gross plot area or before seeking Occupation
tthhee sseeggrre<span style='color:red'>e</span>ggaattiinngg ddiissttaanncce<span style='color:red'>e</span> sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e </span>mmaaiinnttaaiinneedd ffrroomm tthhee eexxiissttiinngg Certificate to any part of building/ buildings beyond 75% of admissible
iinndduussttrriiaall uunniitt.. BUA<span style='color:red'>,</span> other than the built-up amenity.
((22)) AAnnyy pplloott//llaayyoouutt hhaavviinngg<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa eexxcclluuddiinngg<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa uunnddeerr<span style='color:red'> </span>DDPP RRooaadd//
pprreessccrriibbeedd<span style='color:red'> </span>RRLL<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr MMMMCC AAcctt mmaayy<span style='color:red'> </span>bbe<span style='color:red'>e a</span>alllloowweedd ttoo<span style='color:red'> </span>bbe<span style='color:red'>e</span> ddeevveellooppeedd uunnddeerr
(7) The developer shall be entitled to the cost of construction of
tthhiiss RReegguullaattiioonn<span style='color:red'> </span>bbyy iinnssiissttiinngg hhaannddiinngg oovveerr ooff 3355%%<span style='color:red'> a</span>ammeenniittyy ooff tthhee
development of buildable amenity as per provision of regulation 17(1)
rre<span style='color:red'>e</span>qquuiirreedd uunnddeerr RReegguullaattiioonn NNoo.. 1144((BB)) ((cc))..
Note (d).
((33)) RRe<span style='color:red'>e</span>qquuiirreemmeenntt ooff<span style='color:red'> </span>LLOOSS<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthhee pprroovviissiioonnss ooff RReegguullaattiioonn NNoo.. 2277((11))((aa))
sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> iinnssiisstteedd..
(E) Metro Station inter changes: The buildable reservations if any<span style='color:red'>,</span> on
((44)) IInn tthhee eevveenntt ooff<span style='color:red'> </span>DDPP hhaavviinngg pprroovviiddeedd rreesseerrvvaattiioonn//rreesseerrvvaattiioonnss oonn<span style='color:red'> a</span>a pplloott plots abutting Metro Station interchanges shall cease to exist on
wwhheerre<span style='color:red'>e</span> ddeevveellooppmmeenntt uunnddeerr RReegguullaattiioonn NNoo.. 3333((1100)) iiss pprrooppoosseedd<span style='color:red'>,,</span> tthhee<span style='color:red'>
</span>identification of such Metro Station interchanges within 500 m. from
ffoolllloowwiinngg sshhaallll<span style='color:red'> a</span>appppllyy..
such interchange to that extent<span style='color:red'>,</span> subject to condition that 20% of plot<span style='color:red'>
a</span>a.. IIff tthhee<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa uunnddeerr<span style='color:red'> </span>DDPP rreesseerrvvaattiioonn ttoo<span style='color:red'> </span>bbe<span style='color:red'>e</span> hhaannddeedd oovveerr ttoo MMCCGGMM area shall be kept reserved<span style='color:red'> for</span> parking lot<span style='color:red'>,</span> which<span style='color:red'> can</span> be developed
((eexxcclluuddiinngg tthhee<span style='color:red'> a</span>arre<span style='color:red'>e</span>aass uunnddeerr<span style='color:red'> </span>DDPP<span style='color:red'> </span>rrooaaddss//<span style='color:red'> </span>rrooaadd<span style='color:red'> </span>sseettbbaacckk)) iiss lle<span style='color:red'>e</span>ssss tthhaann under accommodation reservation policy under regulation 17(1).
tthhee<span style='color:red'> </span>llaanndd<span style='color:red'> </span>ccoommppoonneenntt ooff bbuuiilltt--uupp<span style='color:red'> a</span>ammeenniittyy rre<span style='color:red'>e</span>qquuiirreedd<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthhiiss However<span style='color:red'>,</span> the non- buildable (open spaces) reservations shall be
RReegguullaattiioonn<span style='color:red'>,,</span> oonnllyy tthhee aaddddiittiioonnaall<span style='color:red'> </span>llaanndd<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span> pprroovviiddeedd ffoorr implemented to the full extent.<span style='color:red'>
a</span>ammeenniittyy..<span style='color:red'>
</span>bb.. IIff tthhee<span style='color:red'> a</span>arre<span style='color:red'>e</span>aa uunnddeerr<span style='color:red'> </span>DDPP rreesseerrvvaattiioonn ttoo<span style='color:red'> </span>bbe<span style='color:red'>e</span> hhaannddeedd oovveerr ttoo MMCCGGMM<span style='color:red'>,,</span>
((eexxcclluuddiinngg tthhee<span style='color:red'> a</span>arre<span style='color:red'>e</span>aass uunnddeerr<span style='color:red'> </span>DDPP<span style='color:red'> </span>rrooaaddss//<span style='color:red'> </span>rrooaadd<span style='color:red'> </span>sseettbbaacckk)),<span style='color:red'>,</span> iiss mmoorre<span style='color:red'>e</span> tthhaann
tthhee<span style='color:red'> </span>llaanndd<span style='color:red'> </span>ccoommppoonneenntt ooff bbuuiilltt--uupp<span style='color:red'> a</span>ammeenniittyy rre<span style='color:red'>e</span>qquuiirreedd<span style='color:red'> a</span>ass<span style='color:red'> </span>ppeerr tthhiiss
RReegguullaattiioonn<span style='color:red'>,,</span> tthheenn tthhee pprroovviissiioonn ffoorr<span style='color:red'> a</span>ammeenniittyy iiss nnoott<span style='color:red'> </span>nne<span style='color:red'>e</span>cceessssaarryy..
((55)) TThhee oowwnneerr sshhaallll<span style='color:red'> </span>ggiivve<span style='color:red'>e</span> aaddvvaanncceedd<span style='color:red'> </span>ppoosssseessssiioonn ooff tthhee<span style='color:red'> </span>llaanndd wwhheerreevveerr<span style='color:red'>
a</span>apppplliiccaabblle<span style='color:red'>e</span> ((ttoo<span style='color:red'> </span>bbe<span style='color:red'>e</span> hhaannddeedd oovveerr)) ttoo MMCCGGMM//AApppprroopprriiaatte<span style='color:red'>e</span> AAuutthhoorriittyy<span style='color:red'> a</span>att tthhee<span style='color:red'>
</span>ttiimme<span style='color:red'>e</span> ooff sseeeekkiinngg AApppprroovvaall ffoorr DDeevveellooppmmeenntt ooff pplloott.. IItt sshhaallll<span style='color:red'> </span>bbe<span style='color:red'>e</span>
rreessppoonnssiibbiilliittyy ooff tthhee<span style='color:red'> </span>llaanndd oowwnneerr ttoo<span style='color:red'> </span>cclle<span style='color:red'>e</span>aarr<span style='color:red'> a</span>allll tthhee eennccuummbbrraanncceess aanndd<span style='color:red'>
</span>ccoommpplleette<span style='color:red'>e</span> tthhee ffoorrmmaalliittiieess<span style='color:red'> </span>ttoowwaarrddss<span style='color:red'> </span>ttrraannssffeerrrriinngg tthhee<span style='color:red'> </span>llaanndd iinn tthhee<span style='color:red'> </span>nnaamme<span style='color:red'>e</span>
110000 DCPR 2034 101 DCPR 2034<span style='color:red'>
</span>

In [ ]:
final_template = f"""
You are an Legal expert Real Estate Agent assistant providing detailed and accurate information. The user has asked the following question:
**Question:** {query}

Below is relevant information retrieved from various sources:
**Retrieved Information:**
{hybrid_results}

**Instruction:** Use only the query-relevant content from the retrieved information to answer the question. Focus on providing a detailed comprehensive, informative response based solely on the given data. If any conflicting details are present, prioritize the most reliable and consistent information.
"""
final_template

"\nYou are an Legal expert assistant providing detailed and accurate information. The user has asked the following question:\n**Question:** Our land is a leese land , society formed , can we apply for deem conveyance\n\nBelow is relevant information retrieved from various sources:\n**Retrieved Information:**\n['Original Notifications shall\\nbe referred for perspicuity\\nvest, by provision in a deed of conveyance, in all the owners on\\naccount of whose cumulative holdings the LOS is required to be\\nkept as LOS in the layout or sub-division/amalgamation/plot of the\\nland.\\n(b) The proposal for construction of such structure should come as a\\nproposal from the owner/owners/society/societies or federation of\\nsocieties shall be meant for the beneficial use of the owner/owners/\\nmembers of such society/societies/federation of societies.\\n(c) Such structures shall not be used for any other purpose, except for\\nrecreational activities.\\n(d) The remaining area of the LOS shall be ke

In [71]:
from groq import Groq
client = Groq(
    api_key='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' ,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": final_template,
        }
    ],
    model="llama-3.1-70b-versatile",
)
print(chat_completion.choices[0].message.content)

**Deemed Conveyance on Lease Land with a Formed Society: A Detailed Analysis**

The concept of deemed conveyance on lease land with a formed society is an important aspect of real estate development in India, particularly in states like Maharashtra. Deemed conveyance is a legal process that grants the lessee (lease holder) the rights equivalent to ownership of the property.

**Eligibility Criteria:**

Based on the retrieved information, it appears that a lease land with a formed society can apply for deemed conveyance. The society must have been formed and registered under the relevant laws, such as the Maharashtra Co-operative Societies Act, 1960. The lease agreement must also have been executed and registered in favor of the society.

**Procedure:**

The procedure for obtaining deemed conveyance on lease land with a formed society involves several steps:

1. **Application:** The society must submit an application to the competent authority, such as the Commissioner of the Municipal C